In [4]:
!pip install torch-geometric


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import add_self_loops, to_networkx
import networkx as nx
import random

# Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')

# Define a simple GCN model
class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 16)
        self.conv2 = GCNConv(16, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Load data
data = dataset[0]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(dataset.num_node_features, dataset.num_classes).to(device)
data = data.to(device)

# Train the GCN model
def train(model, data):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    model.train()

    for epoch in range(200):
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

# Evaluate the model
def test(model, data):
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    acc = int(correct) / int(data.test_mask.sum())
    return acc

# Backdoor Attack Injection (Poisoning)
def inject_backdoor(data, num_poisoned_nodes=5):
    print(f"Injecting backdoor attack into {num_poisoned_nodes} nodes")
    poisoned_nodes = random.sample(range(data.num_nodes), num_poisoned_nodes)
    for node in poisoned_nodes:
        data.x[node] = torch.rand_like(data.x[node])  # Randomize features as trigger
        data.y[node] = random.randint(0, dataset.num_classes - 1)  # Mislabel
    return data, poisoned_nodes

# Anomaly Detection (Simple Defense)
def detect_anomalous_nodes(data):
    G = to_networkx(data, to_undirected=True)
    degrees = nx.degree_centrality(G)

    # Find nodes with degree centrality significantly higher than others
    threshold = 2 * sum(degrees.values()) / len(degrees)
    anomalous_nodes = [node for node, degree in degrees.items() if degree > threshold]
    print(f"Detected {len(anomalous_nodes)} anomalous nodes")
    return anomalous_nodes

# Main execution
print("Training the model on clean data...")
train(model, data)
acc_clean = test(model, data)
print(f"Accuracy on clean data: {acc_clean:.4f}")

# Inject backdoor attacks
data, poisoned_nodes = inject_backdoor(data, num_poisoned_nodes=5)

print("Training the model on poisoned data...")
train(model, data)
acc_poisoned = test(model, data)
print(f"Accuracy on poisoned data: {acc_poisoned:.4f}")

# Detect and filter anomalous nodes
anomalous_nodes = detect_anomalous_nodes(data)

# Fine-tune the model without poisoned nodes
def remove_poisoned_nodes(data, poisoned_nodes):
    data.train_mask[poisoned_nodes] = False  # Remove poisoned nodes from the training set
    return data

data = remove_poisoned_nodes(data, poisoned_nodes)
print("Retraining the model after removing poisoned nodes...")
train(model, data)
acc_defended = test(model, data)
print(f"Accuracy after defense: {acc_defended:.4f}")


Processing...
Done!


Training the model on clean data...
Epoch 0, Loss: 1.9546961784362793
Epoch 10, Loss: 0.6961610913276672
Epoch 20, Loss: 0.19654710590839386
Epoch 30, Loss: 0.0737478956580162
Epoch 40, Loss: 0.07014885544776917
Epoch 50, Loss: 0.04678536579012871
Epoch 60, Loss: 0.04055462405085564
Epoch 70, Loss: 0.04191984608769417
Epoch 80, Loss: 0.04423925653100014
Epoch 90, Loss: 0.04605355113744736
Epoch 100, Loss: 0.03145918250083923
Epoch 110, Loss: 0.02058337815105915
Epoch 120, Loss: 0.035183221101760864
Epoch 130, Loss: 0.02580614574253559
Epoch 140, Loss: 0.03164217993617058
Epoch 150, Loss: 0.024084553122520447
Epoch 160, Loss: 0.03014778345823288
Epoch 170, Loss: 0.02581222914159298
Epoch 180, Loss: 0.03662600368261337
Epoch 190, Loss: 0.03895122930407524
Accuracy on clean data: 0.7980
Injecting backdoor attack into 5 nodes
Training the model on poisoned data...
Epoch 0, Loss: 0.09548074007034302
Epoch 10, Loss: 0.03162235766649246
Epoch 20, Loss: 0.02893128991127014
Epoch 30, Loss: 0.02

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import to_networkx, add_self_loops, remove_self_loops
import random
import networkx as nx
from sklearn.decomposition import PCA
import numpy as np

# Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')

# Define a simple GCN model
class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 16)
        self.conv2 = GCNConv(16, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Load data
data = dataset[0]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(dataset.num_node_features, dataset.num_classes).to(device)
data = data.to(device)

# Train the GCN model
def train(model, data):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    model.train()

    for epoch in range(200):
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

# Evaluate the model
def test(model, data):
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    acc = int(correct) / int(data.test_mask.sum())
    return acc

# Backdoor Attack 1: Feature-based Poisoning (Add triggers to node features)
def inject_feature_attack(data, num_poisoned_nodes=5):
    print(f"Injecting feature backdoor attack into {num_poisoned_nodes} nodes")
    poisoned_nodes = random.sample(range(data.num_nodes), num_poisoned_nodes)
    for node in poisoned_nodes:
        data.x[node] = torch.rand_like(data.x[node])  # Randomize features as a trigger
        data.y[node] = random.randint(0, dataset.num_classes - 1)  # Mislabel
    return data, poisoned_nodes

# Backdoor Attack 2: Edge-based Poisoning (Add random edges)
def inject_edge_attack(data, num_poisoned_edges=10):
    print(f"Injecting edge backdoor attack with {num_poisoned_edges} edges")
    edge_index, _ = remove_self_loops(data.edge_index)
    poisoned_edges = []
    for _ in range(num_poisoned_edges):
        node1 = random.randint(0, data.num_nodes - 1)
        node2 = random.randint(0, data.num_nodes - 1)
        poisoned_edges.append([node1, node2])
    poisoned_edges = torch.tensor(poisoned_edges, dtype=torch.long).t().contiguous()
    data.edge_index = torch.cat([edge_index, poisoned_edges], dim=1)
    return data, poisoned_edges

# Backdoor Attack 3: Trigger-based Poisoning (Feature + Edge modification)
def inject_trigger_attack(data, num_trigger_nodes=5, trigger_pattern=None):
    print(f"Injecting trigger-based backdoor attack into {num_trigger_nodes} nodes")
    poisoned_nodes = random.sample(range(data.num_nodes), num_trigger_nodes)
    if trigger_pattern is None:
        trigger_pattern = torch.ones(data.num_node_features)

    for node in poisoned_nodes:
        data.x[node] = trigger_pattern  # Set a specific trigger pattern in features
        data.y[node] = random.randint(0, dataset.num_classes - 1)  # Mislabel

    # Add edges between poisoned nodes (trigger behavior)
    for i in range(num_trigger_nodes):
        for j in range(i + 1, num_trigger_nodes):
            data.edge_index = torch.cat([data.edge_index, torch.tensor([[poisoned_nodes[i]], [poisoned_nodes[j]]], dtype=torch.long)], dim=1)

    return data, poisoned_nodes

# Filtration Method 1: Community Detection-Based Filtration
def community_detection_filter(data):
    G = to_networkx(data, to_undirected=True)
    communities = nx.algorithms.community.greedy_modularity_communities(G)
    filtered_nodes = []

    # Assuming smaller communities are potentially poisoned
    for community in communities:
        if len(community) < 10:  # Arbitrary threshold
            filtered_nodes.extend(list(community))

    print(f"Community detection identified {len(filtered_nodes)} potentially poisoned nodes")
    return filtered_nodes

# Filtration Method 2: PCA Embedding-Based Detection
def pca_embedding_filter(data, n_components=2):
    X = data.x.cpu().detach().numpy()
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X)

    # Compute distances from the mean of embeddings
    distances = np.linalg.norm(X_pca - X_pca.mean(axis=0), axis=1)
    threshold = np.mean(distances) + 2 * np.std(distances)  # Arbitrary threshold
    filtered_nodes = np.where(distances > threshold)[0]

    print(f"PCA-based embedding filter identified {len(filtered_nodes)} potentially poisoned nodes")
    return filtered_nodes

# Filtration Method 3: Spectral Analysis (Graph Laplacian)
def spectral_analysis_filter(data):
    G = to_networkx(data, to_undirected=True)
    laplacian_matrix = nx.laplacian_matrix(G).toarray()
    eigenvalues = np.linalg.eigvals(laplacian_matrix)

    # Identify anomalies by analyzing the spectrum of eigenvalues
    threshold = np.mean(eigenvalues) + 2 * np.std(eigenvalues)  # Arbitrary threshold
    filtered_nodes = [i for i, e in enumerate(eigenvalues) if e > threshold]

    print(f"Spectral analysis identified {len(filtered_nodes)} potentially poisoned nodes")
    return filtered_nodes

# Main Execution

# Train on clean data
print("Training the model on clean data...")
train(model, data)
acc_clean = test(model, data)
print(f"Accuracy on clean data: {acc_clean:.4f}")

# Inject feature-based backdoor attack
data, poisoned_nodes_feat = inject_feature_attack(data, num_poisoned_nodes=5)

# Inject edge-based backdoor attack
data, poisoned_edges = inject_edge_attack(data, num_poisoned_edges=10)

# Inject trigger-based backdoor attack
trigger_pattern = torch.ones(data.num_node_features) * 0.5  # Custom trigger pattern
data, poisoned_nodes_trigger = inject_trigger_attack(data, num_trigger_nodes=5, trigger_pattern=trigger_pattern)

# Retrain on poisoned data
print("Training the model on poisoned data...")
train(model, data)
acc_poisoned = test(model, data)
print(f"Accuracy on poisoned data: {acc_poisoned:.4f}")

# Apply community detection filtration
community_filtered_nodes = community_detection_filter(data)

# Apply PCA embedding-based filtration
pca_filtered_nodes = pca_embedding_filter(data)

# Apply spectral analysis filtration
spectral_filtered_nodes = spectral_analysis_filter(data)

# Final Retraining after filtering poisoned nodes
def remove_filtered_nodes(data, filtered_nodes):
    data.train_mask[filtered_nodes] = False  # Remove poisoned nodes from training
    return data

# Combine all filtered nodes and retrain
all_filtered_nodes = set(community_filtered_nodes).union(pca_filtered_nodes).union(spectral_filtered_nodes)
data = remove_filtered_nodes(data, list(all_filtered_nodes))

print("Retraining the model after filtering potentially poisoned nodes...")
train(model, data)
acc_defended = test(model, data)
print(f"Accuracy after defense: {acc_defended:.4f}")


Training the model on clean data...
Epoch 0, Loss: 1.9507275819778442
Epoch 10, Loss: 0.7671340703964233
Epoch 20, Loss: 0.23847250640392303
Epoch 30, Loss: 0.14077916741371155
Epoch 40, Loss: 0.06796666234731674
Epoch 50, Loss: 0.05183885619044304
Epoch 60, Loss: 0.057986531406641006
Epoch 70, Loss: 0.03404968976974487
Epoch 80, Loss: 0.037949733436107635
Epoch 90, Loss: 0.03004702925682068
Epoch 100, Loss: 0.030838390812277794
Epoch 110, Loss: 0.031211528927087784
Epoch 120, Loss: 0.04458938166499138
Epoch 130, Loss: 0.02715941146016121
Epoch 140, Loss: 0.046850357204675674
Epoch 150, Loss: 0.026798702776432037
Epoch 160, Loss: 0.02269257791340351
Epoch 170, Loss: 0.03319086879491806
Epoch 180, Loss: 0.029623253270983696
Epoch 190, Loss: 0.023200623691082
Accuracy on clean data: 0.8080
Injecting feature backdoor attack into 5 nodes
Injecting edge backdoor attack with 10 edges
Injecting trigger-based backdoor attack into 5 nodes
Training the model on poisoned data...
Epoch 0, Loss: 0.

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import to_networkx, add_self_loops, remove_self_loops
import random
import networkx as nx
from sklearn.decomposition import PCA
import numpy as np

# Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')

# Define a simple GCN model
class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 16)
        self.conv2 = GCNConv(16, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Load data
data = dataset[0]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(dataset.num_node_features, dataset.num_classes).to(device)
data = data.to(device)

# Train the GCN model
def train(model, data):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    model.train()

    for epoch in range(200):
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

# Evaluate the model
def test(model, data):
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    acc = int(correct) / int(data.test_mask.sum())
    return acc

# Backdoor Attack 1: Feature-based Poisoning
def inject_feature_attack(data, num_poisoned_nodes=5):
    print(f"Injecting feature backdoor attack into {num_poisoned_nodes} nodes")
    poisoned_nodes = random.sample(range(data.num_nodes), num_poisoned_nodes)
    for node in poisoned_nodes:
        data.x[node] = torch.rand_like(data.x[node])  # Randomize features as a trigger
        data.y[node] = random.randint(0, dataset.num_classes - 1)  # Mislabel
    return data, poisoned_nodes

# Backdoor Attack 2: Edge-based Poisoning
def inject_edge_attack(data, num_poisoned_edges=10):
    print(f"Injecting edge backdoor attack with {num_poisoned_edges} edges")
    edge_index, _ = remove_self_loops(data.edge_index)
    poisoned_edges = []
    for _ in range(num_poisoned_edges):
        node1 = random.randint(0, data.num_nodes - 1)
        node2 = random.randint(0, data.num_nodes - 1)
        poisoned_edges.append([node1, node2])
    poisoned_edges = torch.tensor(poisoned_edges, dtype=torch.long).t().contiguous()
    data.edge_index = torch.cat([edge_index, poisoned_edges], dim=1)
    return data, poisoned_edges

# Filtration Method 1: Community Detection-Based Filtration
def community_detection_filter(data):
    G = to_networkx(data, to_undirected=True)
    communities = nx.algorithms.community.greedy_modularity_communities(G)
    filtered_nodes = []

    # Assuming smaller communities are potentially poisoned
    for community in communities:
        if len(community) < 10:  # Arbitrary threshold
            filtered_nodes.extend(list(community))

    print(f"Community detection identified {len(filtered_nodes)} potentially poisoned nodes")
    return filtered_nodes

# Filtration Method 2: PCA Embedding-Based Detection
def pca_embedding_filter(data, n_components=2):
    X = data.x.cpu().detach().numpy()
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X)

    # Compute distances from the mean of embeddings
    distances = np.linalg.norm(X_pca - X_pca.mean(axis=0), axis=1)
    threshold = np.mean(distances) + 2 * np.std(distances)  # Arbitrary threshold
    filtered_nodes = np.where(distances > threshold)[0]

    print(f"PCA-based embedding filter identified {len(filtered_nodes)} potentially poisoned nodes")
    return filtered_nodes

# Final Retraining after filtering poisoned nodes
def remove_filtered_nodes(data, filtered_nodes):
    data.train_mask[filtered_nodes] = False  # Remove poisoned nodes from training
    return data

# Apply backdoor attacks and filtration one by one

# 1. Apply Feature-based Attack and Community Detection
print("===== Feature-based Attack and Community Detection Filtration =====")
data, poisoned_nodes_feat = inject_feature_attack(data, num_poisoned_nodes=5)
print("Training the model on poisoned data...")
train(model, data)
acc_poisoned_feat = test(model, data)
print(f"Accuracy on poisoned data (feature-based attack): {acc_poisoned_feat:.4f}")

# Apply community detection filter
community_filtered_nodes = community_detection_filter(data)
data = remove_filtered_nodes(data, community_filtered_nodes)
print("Retraining the model after community detection filtering...")
train(model, data)
acc_defended_feat = test(model, data)
print(f"Accuracy after community detection filter (feature-based attack): {acc_defended_feat:.4f}")

# 2. Apply Edge-based Attack and PCA Embedding Filtration
print("\n===== Edge-based Attack and PCA Embedding Filtration =====")
data, poisoned_edges = inject_edge_attack(data, num_poisoned_edges=10)
print("Training the model on poisoned data...")
train(model, data)
acc_poisoned_edge = test(model, data)
print(f"Accuracy on poisoned data (edge-based attack): {acc_poisoned_edge:.4f}")

# Apply PCA embedding-based filter
pca_filtered_nodes = pca_embedding_filter(data)
data = remove_filtered_nodes(data, pca_filtered_nodes)
print("Retraining the model after PCA embedding filtering...")
train(model, data)
acc_defended_edge = test(model, data)
print(f"Accuracy after PCA embedding filter (edge-based attack): {acc_defended_edge:.4f}")

# 3. Apply Trigger-based Attack and Spectral Analysis
print("\n===== Trigger-based Attack and Spectral Analysis Filtration =====")
trigger_pattern = torch.ones(data.num_node_features) * 0.5  # Custom trigger pattern
data, poisoned_nodes_trigger = inject_feature_attack(data, num_poisoned_nodes=5)
print("Training the model on poisoned data...")
train(model, data)
acc_poisoned_trigger = test(model, data)
print(f"Accuracy on poisoned data (trigger-based attack): {acc_poisoned_trigger:.4f}")

# Apply spectral analysis filtration
def spectral_analysis_filter(data):
    G = to_networkx(data, to_undirected=True)
    laplacian_matrix = nx.laplacian_matrix(G).toarray()
    eigenvalues = np.linalg.eigvals(laplacian_matrix)

    # Identify anomalies by analyzing the spectrum of eigenvalues
    threshold = np.mean(eigenvalues) + 2 * np.std(eigenvalues)  # Arbitrary threshold
    filtered_nodes = [i for i, e in enumerate(eigenvalues) if e > threshold]

    print(f"Spectral analysis identified {len(filtered_nodes)} potentially poisoned nodes")
    return filtered_nodes

spectral_filtered_nodes = spectral_analysis_filter(data)
data = remove_filtered_nodes(data, spectral_filtered_nodes)
print("Retraining the model after spectral analysis filtering...")
train(model, data)
acc_defended_trigger = test(model, data)
print(f"Accuracy after spectral analysis filter (trigger-based attack): {acc_defended_trigger:.4f}")


===== Feature-based Attack and Community Detection Filtration =====
Injecting feature backdoor attack into 5 nodes
Training the model on poisoned data...
Accuracy on poisoned data (feature-based attack): 0.8100
Community detection identified 245 potentially poisoned nodes
Retraining the model after community detection filtering...
Accuracy after community detection filter (feature-based attack): 0.7930

===== Edge-based Attack and PCA Embedding Filtration =====
Injecting edge backdoor attack with 10 edges
Training the model on poisoned data...
Accuracy on poisoned data (edge-based attack): 0.7700
PCA-based embedding filter identified 5 potentially poisoned nodes
Retraining the model after PCA embedding filtering...
Accuracy after PCA embedding filter (edge-based attack): 0.7640

===== Trigger-based Attack and Spectral Analysis Filtration =====
Injecting feature backdoor attack into 5 nodes
Training the model on poisoned data...
Accuracy on poisoned data (trigger-based attack): 0.7940
S

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import to_networkx, add_self_loops, remove_self_loops
import random
import networkx as nx
from sklearn.decomposition import PCA
import numpy as np

# Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')

# Define a simple GCN model
class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 16)
        self.conv2 = GCNConv(16, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Load data
original_data = dataset[0]  # Save the original dataset separately

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the GCN model
def train(model, data):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    model.train()

    for epoch in range(200):
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

# Evaluate the model
def test(model, data):
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    acc = int(correct) / int(data.test_mask.sum())
    return acc

# Backdoor Attack 1: Feature-based Poisoning
def inject_feature_attack(data, num_poisoned_nodes=5):
    poisoned_nodes = random.sample(range(data.num_nodes), num_poisoned_nodes)
    for node in poisoned_nodes:
        data.x[node] = torch.rand_like(data.x[node])  # Randomize features as a trigger
        data.y[node] = random.randint(0, dataset.num_classes - 1)  # Mislabel
    return data, poisoned_nodes

# Backdoor Attack 2: Edge-based Poisoning
def inject_edge_attack(data, num_poisoned_edges=10):
    edge_index, _ = remove_self_loops(data.edge_index)
    poisoned_edges = []
    for _ in range(num_poisoned_edges):
        node1 = random.randint(0, data.num_nodes - 1)
        node2 = random.randint(0, data.num_nodes - 1)
        poisoned_edges.append([node1, node2])
    poisoned_edges = torch.tensor(poisoned_edges, dtype=torch.long).t().contiguous()
    data.edge_index = torch.cat([edge_index, poisoned_edges], dim=1)
    return data, poisoned_edges

# Backdoor Attack 3: Trigger-based Poisoning
def inject_trigger_attack(data, num_trigger_nodes=5, trigger_pattern=None):
    poisoned_nodes = random.sample(range(data.num_nodes), num_trigger_nodes)
    if trigger_pattern is None:
        trigger_pattern = torch.ones(data.num_node_features)

    for node in poisoned_nodes:
        data.x[node] = trigger_pattern  # Set a specific trigger pattern in features
        data.y[node] = random.randint(0, dataset.num_classes - 1)  # Mislabel

    return data, poisoned_nodes

# Filtration Method 1: Community Detection-Based Filtration
def community_detection_filter(data):
    G = to_networkx(data, to_undirected=True)
    communities = nx.algorithms.community.greedy_modularity_communities(G)
    filtered_nodes = []

    # Assuming smaller communities are potentially poisoned
    for community in communities:
        if len(community) < 10:  # Arbitrary threshold
            filtered_nodes.extend(list(community))

    return filtered_nodes

# Filtration Method 2: PCA Embedding-Based Detection
def pca_embedding_filter(data, n_components=2):
    X = data.x.cpu().detach().numpy()
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X)

    # Compute distances from the mean of embeddings
    distances = np.linalg.norm(X_pca - X_pca.mean(axis=0), axis=1)
    threshold = np.mean(distances) + 2 * np.std(distances)  # Arbitrary threshold
    filtered_nodes = np.where(distances > threshold)[0]

    return filtered_nodes

# Filtration Method 3: Spectral Analysis
def spectral_analysis_filter(data):
    G = to_networkx(data, to_undirected=True)
    laplacian_matrix = nx.laplacian_matrix(G).toarray()
    eigenvalues = np.linalg.eigvals(laplacian_matrix)

    # Identify anomalies by analyzing the spectrum of eigenvalues
    threshold = np.mean(eigenvalues) + 2 * np.std(eigenvalues)  # Arbitrary threshold
    filtered_nodes = [i for i, e in enumerate(eigenvalues) if e > threshold]

    return filtered_nodes

# Final Retraining after filtering poisoned nodes
def remove_filtered_nodes(data, filtered_nodes):
    data.train_mask[filtered_nodes] = False  # Remove poisoned nodes from training
    return data

# Train on clean data
def run_clean_model():
    print("Training the model on clean data...")
    data = original_data.clone().to(device)  # Reset to original clean data
    model = GCN(dataset.num_node_features, dataset.num_classes).to(device)
    train(model, data)
    acc_clean = test(model, data)
    print(f"Clean Accuracy: {acc_clean:.4f}")
    return acc_clean

# Function to execute one case
def run_case(attack_func, filter_func, attack_name, filter_name):
    print(f"\n===== {attack_name} and {filter_name} Filtration =====")

    # Step 1: Reset to original clean data before attack
    data = original_data.clone().to(device)
    model = GCN(dataset.num_node_features, dataset.num_classes).to(device)

    # Step 2: Apply Backdoor Attack
    data, poisoned_nodes = attack_func(data)
    print(f"Training the model on poisoned data ({attack_name})...")
    train(model, data)
    acc_poisoned = test(model, data)
    print(f"Accuracy on poisoned data ({attack_name}): {acc_poisoned:.4f}")

    # Step 3: Apply Filtration
    filtered_nodes = filter_func(data)
    data = remove_filtered_nodes(data, filtered_nodes)

    print(f"Retraining the model after {filter_name} filtering...")
    train(model, data)
    acc_defended = test(model, data)
    print(f"Accuracy after {filter_name} filter ({attack_name}): {acc_defended:.4f}")

# Running all 9 cases
attacks = [(inject_feature_attack, "Feature-based Attack"),
           (inject_edge_attack, "Edge-based Attack"),
           (inject_trigger_attack, "Trigger-based Attack")]

filters = [(community_detection_filter, "Community Detection"),
           (pca_embedding_filter, "PCA Embedding"),
           (spectral_analysis_filter, "Spectral Analysis")]

# First, show the clean accuracy
clean_accuracy = run_clean_model()

# Now, run all 9 combinations of attacks and filtrations
for attack_func, attack_name in attacks:
    for filter_func, filter_name in filters:
        run_case(attack_func, filter_func, attack_name, filter_name)


Training the model on clean data...
Clean Accuracy: 0.8030

===== Feature-based Attack and Community Detection Filtration =====
Training the model on poisoned data (Feature-based Attack)...
Accuracy on poisoned data (Feature-based Attack): 0.8060
Retraining the model after Community Detection filtering...
Accuracy after Community Detection filter (Feature-based Attack): 0.7920

===== Feature-based Attack and PCA Embedding Filtration =====
Training the model on poisoned data (Feature-based Attack)...
Accuracy on poisoned data (Feature-based Attack): 0.7920
Retraining the model after PCA Embedding filtering...
Accuracy after PCA Embedding filter (Feature-based Attack): 0.8080

===== Feature-based Attack and Spectral Analysis Filtration =====
Training the model on poisoned data (Feature-based Attack)...
Accuracy on poisoned data (Feature-based Attack): 0.8030
Retraining the model after Spectral Analysis filtering...
Accuracy after Spectral Analysis filter (Feature-based Attack): 0.5610

=

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import to_networkx, remove_self_loops
import random
import networkx as nx
from sklearn.decomposition import PCA
import numpy as np

# Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')

# Define a simple GCN model
class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 16)
        self.conv2 = GCNConv(16, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Load data
original_data = dataset[0]  # Save the original dataset separately

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the GCN model
def train(model, data):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    model.train()

    for epoch in range(200):
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

# Evaluate the model
def test(model, data):
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    acc = int(correct) / int(data.test_mask.sum())
    return acc

# Backdoor Attack 1: Feature-based Poisoning
def inject_feature_attack(data, num_poisoned_nodes=5):
    poisoned_nodes = random.sample(range(data.num_nodes), num_poisoned_nodes)
    for node in poisoned_nodes:
        data.x[node] = torch.rand_like(data.x[node])  # Randomize features as a trigger
        data.y[node] = random.randint(0, dataset.num_classes - 1)  # Mislabel
    return data, poisoned_nodes

# Backdoor Attack 2: Edge-based Poisoning
def inject_edge_attack(data, num_poisoned_edges=10):
    edge_index, _ = remove_self_loops(data.edge_index)
    poisoned_edges = []
    for _ in range(num_poisoned_edges):
        node1 = random.randint(0, data.num_nodes - 1)
        node2 = random.randint(0, data.num_nodes - 1)
        poisoned_edges.append([node1, node2])
    poisoned_edges = torch.tensor(poisoned_edges, dtype=torch.long).t().contiguous()
    data.edge_index = torch.cat([edge_index, poisoned_edges], dim=1)
    return data, poisoned_edges

# Backdoor Attack 3: Trigger-based Poisoning
def inject_trigger_attack(data, num_trigger_nodes=5, trigger_pattern=None):
    poisoned_nodes = random.sample(range(data.num_nodes), num_trigger_nodes)
    if trigger_pattern is None:
        trigger_pattern = torch.ones(data.num_node_features)

    for node in poisoned_nodes:
        data.x[node] = trigger_pattern  # Set a specific trigger pattern in features
        data.y[node] = random.randint(0, dataset.num_classes - 1)  # Mislabel

    return data, poisoned_nodes

# Filtration Method 1: Community Detection-Based Filtration
def community_detection_filter(data):
    G = to_networkx(data, to_undirected=True)
    communities = nx.algorithms.community.greedy_modularity_communities(G)
    filtered_nodes = []

    # Assuming smaller communities are potentially poisoned
    for community in communities:
        if len(community) < 10:  # Arbitrary threshold
            filtered_nodes.extend(list(community))

    return filtered_nodes

# Filtration Method 2: PCA Embedding-Based Detection
def pca_embedding_filter(data, n_components=2):
    X = data.x.cpu().detach().numpy()
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X)

    # Compute distances from the mean of embeddings
    distances = np.linalg.norm(X_pca - X_pca.mean(axis=0), axis=1)
    threshold = np.mean(distances) + 2 * np.std(distances)  # Arbitrary threshold
    filtered_nodes = np.where(distances > threshold)[0]

    return filtered_nodes

# Filtration Method 3: Spectral Analysis
def spectral_analysis_filter(data):
    G = to_networkx(data, to_undirected=True)
    laplacian_matrix = nx.laplacian_matrix(G).toarray()
    eigenvalues = np.linalg.eigvals(laplacian_matrix)

    # Identify anomalies by analyzing the spectrum of eigenvalues
    threshold = np.mean(eigenvalues) + 2 * np.std(eigenvalues)  # Arbitrary threshold
    filtered_nodes = [i for i, e in enumerate(eigenvalues) if e > threshold]

    return filtered_nodes

# Final Retraining after filtering poisoned nodes
def remove_filtered_nodes(data, filtered_nodes):
    data.train_mask[filtered_nodes] = False  # Remove poisoned nodes from training
    return data

# Train on clean data
def run_clean_model():
    print("Training the model on clean data...")
    data = original_data.clone().to(device)  # Reset to original clean data
    model = GCN(dataset.num_node_features, dataset.num_classes).to(device)
    train(model, data)
    acc_clean = test(model, data)
    print(f"Clean Accuracy: {acc_clean:.4f}")
    return acc_clean

# Function to execute one case
def run_case(attack_func, filter_func, attack_name, filter_name, num_poisoned):
    print(f"\n===== {attack_name} and {filter_name} Filtration =====")

    # Step 1: Reset to original clean data before attack
    data = original_data.clone().to(device)
    model = GCN(dataset.num_node_features, dataset.num_classes).to(device)

    # Step 2: Apply Backdoor Attack
    data, poisoned_nodes = attack_func(data, num_poisoned)
    print(f"Training the model on poisoned data ({attack_name})...")
    train(model, data)
    acc_poisoned = test(model, data)
    print(f"Accuracy on poisoned data ({attack_name}): {acc_poisoned:.4f}")

    # Step 3: Apply Filtration
    filtered_nodes = filter_func(data)
    num_identified_backdoor = len(set(filtered_nodes) & set(poisoned_nodes))
    percent_removed = (num_identified_backdoor / len(poisoned_nodes)) * 100

    print(f"Backdoor Nodes Identified: {num_identified_backdoor}/{len(poisoned_nodes)}")
    print(f"Percentage of Backdoor Nodes Removed: {percent_removed:.2f}%")

    data = remove_filtered_nodes(data, filtered_nodes)

    print(f"Retraining the model after {filter_name} filtering...")
    train(model, data)
    acc_defended = test(model, data)
    print(f"Accuracy after {filter_name} filter ({attack_name}): {acc_defended:.4f}")
    return acc_defended, num_identified_backdoor, percent_removed

# Running all 9 cases
attacks = [(inject_feature_attack, "Feature-based Attack"),
           (inject_edge_attack, "Edge-based Attack"),
           (inject_trigger_attack, "Trigger-based Attack")]

filters = [(community_detection_filter, "Community Detection"),
           (pca_embedding_filter, "PCA Embedding"),
           (spectral_analysis_filter, "Spectral Analysis")]

# First, show the clean accuracy
clean_accuracy = run_clean_model()

# Now, run all 9 combinations of attacks and filtrations
for attack_func, attack_name in attacks:
    for filter_func, filter_name in filters:
        run_case(attack_func, filter_func, attack_name, filter_name, num_poisoned=5)


Training the model on clean data...
Clean Accuracy: 0.8150

===== Feature-based Attack and Community Detection Filtration =====
Training the model on poisoned data (Feature-based Attack)...
Accuracy on poisoned data (Feature-based Attack): 0.7920
Backdoor Nodes Identified: 0/5
Percentage of Backdoor Nodes Removed: 0.00%
Retraining the model after Community Detection filtering...
Accuracy after Community Detection filter (Feature-based Attack): 0.7870

===== Feature-based Attack and PCA Embedding Filtration =====
Training the model on poisoned data (Feature-based Attack)...
Accuracy on poisoned data (Feature-based Attack): 0.8100
Backdoor Nodes Identified: 5/5
Percentage of Backdoor Nodes Removed: 100.00%
Retraining the model after PCA Embedding filtering...
Accuracy after PCA Embedding filter (Feature-based Attack): 0.8010

===== Feature-based Attack and Spectral Analysis Filtration =====
Training the model on poisoned data (Feature-based Attack)...
Accuracy on poisoned data (Feature-b

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import to_networkx, remove_self_loops
import random
import networkx as nx
from sklearn.decomposition import PCA
import numpy as np

# Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')

# Define a simple GCN model
class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 16)
        self.conv2 = GCNConv(16, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Load data
original_data = dataset[0]  # Save the original dataset separately

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the GCN model
def train(model, data):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    model.train()

    for epoch in range(200):
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

# Evaluate the model
def test(model, data):
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    acc = int(correct) / int(data.test_mask.sum())
    return acc

# Backdoor Attack 1: Feature-based Poisoning
def inject_feature_attack(data, num_poisoned_nodes=5):
    poisoned_nodes = random.sample(range(data.num_nodes), num_poisoned_nodes)
    for node in poisoned_nodes:
        data.x[node] = torch.rand_like(data.x[node])  # Randomize features as a trigger
        data.y[node] = random.randint(0, dataset.num_classes - 1)  # Mislabel
    return data, poisoned_nodes

# Backdoor Attack 2: Edge-based Poisoning
def inject_edge_attack(data, num_poisoned_edges=10):
    edge_index, _ = remove_self_loops(data.edge_index)
    poisoned_edges = []
    for _ in range(num_poisoned_edges):
        node1 = random.randint(0, data.num_nodes - 1)
        node2 = random.randint(0, data.num_nodes - 1)
        poisoned_edges.append([node1, node2])
    poisoned_edges = torch.tensor(poisoned_edges, dtype=torch.long).t().contiguous()
    data.edge_index = torch.cat([edge_index, poisoned_edges], dim=1)
    return data, poisoned_edges

# Backdoor Attack 3: Trigger-based Poisoning
def inject_trigger_attack(data, num_trigger_nodes=5, trigger_pattern=None):
    poisoned_nodes = random.sample(range(data.num_nodes), num_trigger_nodes)
    if trigger_pattern is None:
        trigger_pattern = torch.ones(data.num_node_features)

    for node in poisoned_nodes:
        data.x[node] = trigger_pattern  # Set a specific trigger pattern in features
        data.y[node] = random.randint(0, dataset.num_classes - 1)  # Mislabel

    return data, poisoned_nodes

# Filtration Method 1: Community Detection-Based Filtration
def community_detection_filter(data):
    G = to_networkx(data, to_undirected=True)
    communities = nx.algorithms.community.greedy_modularity_communities(G)
    filtered_nodes = []

    # Assuming smaller communities are potentially poisoned
    for community in communities:
        if len(community) < 10:  # Arbitrary threshold
            filtered_nodes.extend(list(community))

    return filtered_nodes

# Filtration Method 2: PCA Embedding-Based Detection
def pca_embedding_filter(data, n_components=2):
    X = data.x.cpu().detach().numpy()
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X)

    # Compute distances from the mean of embeddings
    distances = np.linalg.norm(X_pca - X_pca.mean(axis=0), axis=1)
    threshold = np.mean(distances) + 2 * np.std(distances)  # Arbitrary threshold
    filtered_nodes = np.where(distances > threshold)[0]

    return filtered_nodes

# Filtration Method 3: Spectral Analysis
def spectral_analysis_filter(data):
    G = to_networkx(data, to_undirected=True)
    laplacian_matrix = nx.laplacian_matrix(G).toarray()
    eigenvalues = np.linalg.eigvals(laplacian_matrix)

    # Identify anomalies by analyzing the spectrum of eigenvalues
    threshold = np.mean(eigenvalues) + 2 * np.std(eigenvalues)  # Arbitrary threshold
    filtered_nodes = [i for i, e in enumerate(eigenvalues) if e > threshold]

    return filtered_nodes

# Final Retraining after filtering poisoned nodes
def remove_filtered_nodes(data, filtered_nodes):
    data.train_mask[filtered_nodes] = False  # Remove poisoned nodes from training
    return data

# Train on clean data
def run_clean_model():
    print("Training the model on clean data...")
    data = original_data.clone().to(device)  # Reset to original clean data
    model = GCN(dataset.num_node_features, dataset.num_classes).to(device)
    train(model, data)
    acc_clean = test(model, data)
    print(f"Clean Accuracy: {acc_clean:.4f}")
    return acc_clean

# Function to execute one case
def run_case(attack_func, filter_func, attack_name, filter_name, num_poisoned):
    print(f"\n===== {attack_name} and {filter_name} Filtration =====")

    # Step 1: Reset to original clean data before attack
    data = original_data.clone().to(device)
    model = GCN(dataset.num_node_features, dataset.num_classes).to(device)

    # Step 2: Apply Backdoor Attack
    data, poisoned_nodes = attack_func(data, num_poisoned)
    print(f"Training the model on poisoned data ({attack_name})...")
    train(model, data)
    acc_poisoned = test(model, data)
    print(f"Accuracy on poisoned data ({attack_name}): {acc_poisoned:.4f}")

    # Step 3: Apply Filtration
    filtered_nodes = filter_func(data)
    num_identified_backdoor = len(set(filtered_nodes) & set(poisoned_nodes))
    percent_removed = (num_identified_backdoor / len(poisoned_nodes)) * 100

    print(f"Backdoor Nodes Identified: {num_identified_backdoor}/{len(poisoned_nodes)}")
    print(f"Percentage of Backdoor Nodes Removed: {percent_removed:.2f}%")

    data = remove_filtered_nodes(data, filtered_nodes)

    print(f"Retraining the model after {filter_name} filtering...")
    train(model, data)
    acc_defended = test(model, data)
    print(f"Accuracy after {filter_name} filter ({attack_name}): {acc_defended:.4f}")
    return acc_defended, num_identified_backdoor, percent_removed

# Function to select the optimal policy based on results
def select_optimal_policy(results):
    best_accuracy = 0
    best_policy = None
    best_removal = 0

    for result in results:
        acc, removed_nodes, _ = result
        if acc > best_accuracy and removed_nodes > best_removal:
            best_accuracy = acc
            best_removal = removed_nodes
            best_policy = result

    print(f"\nOptimal Policy: Accuracy = {best_policy[0]:.4f}, Backdoor Nodes Removed = {best_policy[1]}")
    return best_policy

# Running all attack and filter combinations
attacks = [(inject_feature_attack, "Feature-based Attack"),
           (inject_edge_attack, "Edge-based Attack"),
           (inject_trigger_attack, "Trigger-based Attack")]

filters = [(community_detection_filter, "Community Detection"),
           (pca_embedding_filter, "PCA Embedding"),
           (spectral_analysis_filter, "Spectral Analysis")]

# First, show the clean accuracy
clean_accuracy = run_clean_model()

# Running the new attack and filter combinations and gathering results
results = []
for attack_func, attack_name in attacks:
    for filter_func, filter_name in filters:
        result = run_case(attack_func, filter_func, attack_name, filter_name, num_poisoned=5)
        results.append(result)

# Select and display the optimal policy
optimal_policy = select_optimal_policy(results)


Training the model on clean data...
Clean Accuracy: 0.8040

===== Feature-based Attack and Community Detection Filtration =====
Training the model on poisoned data (Feature-based Attack)...
Accuracy on poisoned data (Feature-based Attack): 0.8060
Backdoor Nodes Identified: 1/5
Percentage of Backdoor Nodes Removed: 20.00%
Retraining the model after Community Detection filtering...
Accuracy after Community Detection filter (Feature-based Attack): 0.7880

===== Feature-based Attack and PCA Embedding Filtration =====
Training the model on poisoned data (Feature-based Attack)...
Accuracy on poisoned data (Feature-based Attack): 0.7980
Backdoor Nodes Identified: 5/5
Percentage of Backdoor Nodes Removed: 100.00%
Retraining the model after PCA Embedding filtering...
Accuracy after PCA Embedding filter (Feature-based Attack): 0.7950

===== Feature-based Attack and Spectral Analysis Filtration =====
Training the model on poisoned data (Feature-based Attack)...
Accuracy on poisoned data (Feature-

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import to_networkx, remove_self_loops
import random
import networkx as nx
from sklearn.decomposition import PCA
import numpy as np

# Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')

# Define a simple GCN model
class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 16)
        self.conv2 = GCNConv(16, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Load data
original_data = dataset[0]  # Save the original dataset separately

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the GCN model
def train(model, data):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    model.train()

    for epoch in range(200):
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

# Evaluate the model
def test(model, data):
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    acc = int(correct) / int(data.test_mask.sum())
    return acc

# Backdoor Attack 1: Feature-based Poisoning
def inject_feature_attack(data, num_poisoned_nodes=5):
    poisoned_nodes = random.sample(range(data.num_nodes), num_poisoned_nodes)
    for node in poisoned_nodes:
        data.x[node] = torch.rand_like(data.x[node])  # Randomize features as a trigger
        data.y[node] = random.randint(0, dataset.num_classes - 1)  # Mislabel
    return data, poisoned_nodes

# Backdoor Attack 2: Edge-based Poisoning
def inject_edge_attack(data, num_poisoned_edges=10):
    edge_index, _ = remove_self_loops(data.edge_index)
    poisoned_edges = []
    for _ in range(num_poisoned_edges):
        node1 = random.randint(0, data.num_nodes - 1)
        node2 = random.randint(0, data.num_nodes - 1)
        poisoned_edges.append([node1, node2])
    poisoned_edges = torch.tensor(poisoned_edges, dtype=torch.long).t().contiguous()
    data.edge_index = torch.cat([edge_index, poisoned_edges], dim=1)
    return data, poisoned_edges

# Backdoor Attack 3: Trigger-based Poisoning
def inject_trigger_attack(data, num_trigger_nodes=5, trigger_pattern=None):
    poisoned_nodes = random.sample(range(data.num_nodes), num_trigger_nodes)
    if trigger_pattern is None:
        trigger_pattern = torch.ones(data.num_node_features)

    for node in poisoned_nodes:
        data.x[node] = trigger_pattern  # Set a specific trigger pattern in features
        data.y[node] = random.randint(0, dataset.num_classes - 1)  # Mislabel

    return data, poisoned_nodes

# Augmentation Method 1: Node Feature Noise
def add_node_feature_noise(data, noise_level=0.1):
    noise = torch.randn_like(data.x) * noise_level
    data.x += noise  # Adding noise to node features
    return data

# Augmentation Method 2: Random Edge Dropout
def random_edge_dropout(data, drop_prob=0.1):
    edge_index, _ = remove_self_loops(data.edge_index)
    num_edges = edge_index.size(1)
    drop_mask = torch.rand(num_edges) > drop_prob  # Drop edges with probability drop_prob
    data.edge_index = edge_index[:, drop_mask]
    return data

# Filtration Method 1: Community Detection-Based Filtration
def community_detection_filter(data):
    G = to_networkx(data, to_undirected=True)
    communities = nx.algorithms.community.greedy_modularity_communities(G)
    filtered_nodes = []

    # Assuming smaller communities are potentially poisoned
    for community in communities:
        if len(community) < 10:  # Arbitrary threshold
            filtered_nodes.extend(list(community))

    return filtered_nodes

# Filtration Method 2: PCA Embedding-Based Detection
def pca_embedding_filter(data, n_components=2):
    X = data.x.cpu().detach().numpy()
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X)

    # Compute distances from the mean of embeddings
    distances = np.linalg.norm(X_pca - X_pca.mean(axis=0), axis=1)
    threshold = np.mean(distances) + 2 * np.std(distances)  # Arbitrary threshold
    filtered_nodes = np.where(distances > threshold)[0]

    return filtered_nodes

# Filtration Method 3: Spectral Analysis
def spectral_analysis_filter(data):
    G = to_networkx(data, to_undirected=True)
    laplacian_matrix = nx.laplacian_matrix(G).toarray()
    eigenvalues = np.linalg.eigvals(laplacian_matrix)

    # Identify anomalies by analyzing the spectrum of eigenvalues
    threshold = np.mean(eigenvalues) + 2 * np.std(eigenvalues)  # Arbitrary threshold
    filtered_nodes = [i for i, e in enumerate(eigenvalues) if e > threshold]

    return filtered_nodes

# Final Retraining after filtering poisoned nodes
def remove_filtered_nodes(data, filtered_nodes):
    data.train_mask[filtered_nodes] = False  # Remove poisoned nodes from training
    return data

# Train on clean data
def run_clean_model():
    print("Training the model on clean data...")
    data = original_data.clone().to(device)  # Reset to original clean data
    model = GCN(dataset.num_node_features, dataset.num_classes).to(device)
    train(model, data)
    acc_clean = test(model, data)
    print(f"Clean Accuracy: {acc_clean:.4f}")
    return acc_clean

# Function to execute one case with augmentation
def run_case(attack_func, augment_func, filter_func, attack_name, augment_name, filter_name, num_poisoned):
    print(f"\n===== {attack_name}, {augment_name}, and {filter_name} Filtration =====")

    # Step 1: Reset to original clean data before attack
    data = original_data.clone().to(device)
    model = GCN(dataset.num_node_features, dataset.num_classes).to(device)

    # Step 2: Apply Backdoor Attack
    data, poisoned_nodes = attack_func(data, num_poisoned)

    # Step 3: Apply Augmentation
    data = augment_func(data)

    print(f"Training the model on poisoned and augmented data ({attack_name} + {augment_name})...")
    train(model, data)
    acc_poisoned = test(model, data)
    print(f"Accuracy on poisoned data ({attack_name} + {augment_name}): {acc_poisoned:.4f}")

    # Step 4: Apply Filtration
    filtered_nodes = filter_func(data)
    num_identified_backdoor = len(set(filtered_nodes) & set(poisoned_nodes))
    percent_removed = (num_identified_backdoor / len(poisoned_nodes)) * 100

    print(f"Backdoor Nodes Identified: {num_identified_backdoor}/{len(poisoned_nodes)}")
    print(f"Percentage of Backdoor Nodes Removed: {percent_removed:.2f}%")

    data = remove_filtered_nodes(data, filtered_nodes)

    print(f"Retraining the model after {filter_name} filtering...")
    train(model, data)
    acc_defended = test(model, data)
    print(f"Accuracy after {filter_name} filter ({attack_name} + {augment_name}): {acc_defended:.4f}")
    return acc_defended, num_identified_backdoor, percent_removed

# Function to select the optimal policy based on results
def select_optimal_policy(results):
    best_accuracy = 0
    best_policy = None
    best_removal = 0

    for result in results:
        acc, removed_nodes, _ = result
        if acc > best_accuracy and removed_nodes > best_removal:
            best_accuracy = acc
            best_removal = removed_nodes
            best_policy = result

    print(f"\nOptimal Policy: Accuracy = {best_policy[0]:.4f}, Backdoor Nodes Removed = {best_policy[1]}")
    return best_policy

# Running all attack, augmentation, and filter combinations
attacks = [(inject_feature_attack, "Feature-based Attack"),
           (inject_edge_attack, "Edge-based Attack"),
           (inject_trigger_attack, "Trigger-based Attack")]

augmentations = [(add_node_feature_noise, "Node Feature Noise"),
                 (random_edge_dropout, "Random Edge Dropout")]

filters = [(community_detection_filter, "Community Detection"),
           (pca_embedding_filter, "PCA Embedding"),
           (spectral_analysis_filter, "Spectral Analysis")]

# First, show the clean accuracy
clean_accuracy = run_clean_model()

# Running the new attack, augmentation, and filter combinations and gathering results
results = []
for attack_func, attack_name in attacks:
    for augment_func, augment_name in augmentations:
        for filter_func, filter_name in filters:
            result = run_case(attack_func, augment_func, filter_func, attack_name, augment_name, filter_name, num_poisoned=5)
            results.append(result)

# Select and display the optimal policy
optimal_policy = select_optimal_policy(results)


Training the model on clean data...
Clean Accuracy: 0.8120

===== Feature-based Attack, Node Feature Noise, and Community Detection Filtration =====
Training the model on poisoned and augmented data (Feature-based Attack + Node Feature Noise)...
Accuracy on poisoned data (Feature-based Attack + Node Feature Noise): 0.7980
Backdoor Nodes Identified: 0/5
Percentage of Backdoor Nodes Removed: 0.00%
Retraining the model after Community Detection filtering...
Accuracy after Community Detection filter (Feature-based Attack + Node Feature Noise): 0.7820

===== Feature-based Attack, Node Feature Noise, and PCA Embedding Filtration =====
Training the model on poisoned and augmented data (Feature-based Attack + Node Feature Noise)...
Accuracy on poisoned data (Feature-based Attack + Node Feature Noise): 0.7740
Backdoor Nodes Identified: 5/5
Percentage of Backdoor Nodes Removed: 100.00%
Retraining the model after PCA Embedding filtering...
Accuracy after PCA Embedding filter (Feature-based Attack

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import to_networkx, remove_self_loops
import random
import networkx as nx
from sklearn.decomposition import PCA
import numpy as np

# Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')

# Define a simple GCN model
class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 16)
        self.conv2 = GCNConv(16, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Load data
original_data = dataset[0]  # Save the original dataset separately

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the GCN model
def train(model, data):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    model.train()

    for epoch in range(200):
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

# Evaluate the model
def test(model, data):
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    acc = int(correct) / int(data.test_mask.sum())
    return acc

# Backdoor Attack 1: Feature-based Poisoning
def inject_feature_attack(data, num_poisoned_nodes=5):
    poisoned_nodes = random.sample(range(data.num_nodes), num_poisoned_nodes)
    for node in poisoned_nodes:
        data.x[node] = torch.rand_like(data.x[node])  # Randomize features as a trigger
        data.y[node] = random.randint(0, dataset.num_classes - 1)  # Mislabel
    return data, poisoned_nodes

# Backdoor Attack 2: Edge-based Poisoning
def inject_edge_attack(data, num_poisoned_edges=10):
    edge_index, _ = remove_self_loops(data.edge_index)
    poisoned_edges = []
    for _ in range(num_poisoned_edges):
        node1 = random.randint(0, data.num_nodes - 1)
        node2 = random.randint(0, data.num_nodes - 1)
        poisoned_edges.append([node1, node2])
    poisoned_edges = torch.tensor(poisoned_edges, dtype=torch.long).t().contiguous()
    data.edge_index = torch.cat([edge_index, poisoned_edges], dim=1)
    return data, poisoned_edges

# Backdoor Attack 3: Trigger-based Poisoning
def inject_trigger_attack(data, num_trigger_nodes=5, trigger_pattern=None):
    poisoned_nodes = random.sample(range(data.num_nodes), num_trigger_nodes)
    if trigger_pattern is None:
        trigger_pattern = torch.ones(data.num_node_features)

    for node in poisoned_nodes:
        data.x[node] = trigger_pattern  # Set a specific trigger pattern in features
        data.y[node] = random.randint(0, dataset.num_classes - 1)  # Mislabel

    return data, poisoned_nodes

# Augmentation Method 1: Node Feature Noise
def add_node_feature_noise(data, noise_level=0.1):
    noise = torch.randn_like(data.x) * noise_level
    data.x += noise  # Adding noise to node features
    return data

# Augmentation Method 2: Random Edge Dropout
def random_edge_dropout(data, drop_prob=0.1):
    edge_index, _ = remove_self_loops(data.edge_index)
    num_edges = edge_index.size(1)
    drop_mask = torch.rand(num_edges) > drop_prob  # Drop edges with probability drop_prob
    data.edge_index = edge_index[:, drop_mask]
    return data

# Filtration Method 1: Community Detection-Based Filtration
def community_detection_filter(data):
    G = to_networkx(data, to_undirected=True)
    communities = nx.algorithms.community.greedy_modularity_communities(G)
    filtered_nodes = []

    # Assuming smaller communities are potentially poisoned
    for community in communities:
        if len(community) < 10:  # Arbitrary threshold
            filtered_nodes.extend(list(community))

    return filtered_nodes

# Filtration Method 2: PCA Embedding-Based Detection
def pca_embedding_filter(data, n_components=2):
    X = data.x.cpu().detach().numpy()
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X)

    # Compute distances from the mean of embeddings
    distances = np.linalg.norm(X_pca - X_pca.mean(axis=0), axis=1)
    threshold = np.mean(distances) + 2 * np.std(distances)  # Arbitrary threshold
    filtered_nodes = np.where(distances > threshold)[0]

    return filtered_nodes

# Filtration Method 3: Spectral Analysis
def spectral_analysis_filter(data):
    G = to_networkx(data, to_undirected=True)
    laplacian_matrix = nx.laplacian_matrix(G).toarray()
    eigenvalues = np.linalg.eigvals(laplacian_matrix)

    # Identify anomalies by analyzing the spectrum of eigenvalues
    threshold = np.mean(eigenvalues) + 2 * np.std(eigenvalues)  # Arbitrary threshold
    filtered_nodes = [i for i, e in enumerate(eigenvalues) if e > threshold]

    return filtered_nodes

# Final Retraining after filtering poisoned nodes
def remove_filtered_nodes(data, filtered_nodes):
    data.train_mask[filtered_nodes] = False  # Remove poisoned nodes from training
    return data

# Train on clean data
def run_clean_model():
    print("Training the model on clean data...")
    data = original_data.clone().to(device)  # Reset to original clean data
    model = GCN(dataset.num_node_features, dataset.num_classes).to(device)
    train(model, data)
    acc_clean = test(model, data)
    print(f"Clean Accuracy: {acc_clean:.4f}")
    return acc_clean

# Function to execute one case with augmentation
def run_case(attack_func, augment_func, filter_func, attack_name, augment_name, filter_name, num_poisoned):
    print(f"\n===== {attack_name}, {augment_name}, and {filter_name} Filtration =====")

    # Step 1: Reset to original clean data before attack
    data = original_data.clone().to(device)
    model = GCN(dataset.num_node_features, dataset.num_classes).to(device)

    # Step 2: Apply Backdoor Attack
    data, poisoned_nodes = attack_func(data, num_poisoned)

    # Step 3: Apply Augmentation
    data = augment_func(data)

    print(f"Training the model on poisoned and augmented data ({attack_name} + {augment_name})...")
    train(model, data)
    acc_poisoned = test(model, data)
    print(f"Accuracy on poisoned data ({attack_name} + {augment_name}): {acc_poisoned:.4f}")

    # Step 4: Apply Filtration
    filtered_nodes = filter_func(data)
    num_identified_backdoor = len(set(filtered_nodes) & set(poisoned_nodes))
    percent_removed = (num_identified_backdoor / len(poisoned_nodes)) * 100

    print(f"Backdoor Nodes Identified: {num_identified_backdoor}/{len(poisoned_nodes)}")
    print(f"Percentage of Backdoor Nodes Removed: {percent_removed:.2f}%")

    data = remove_filtered_nodes(data, filtered_nodes)

    print(f"Retraining the model after {filter_name} filtering...")
    train(model, data)
    acc_defended = test(model, data)
    print(f"Accuracy after {filter_name} filter ({attack_name} + {augment_name}): {acc_defended:.4f}")
    return acc_defended, num_identified_backdoor, percent_removed

# Weighted Scoring Method
def weighted_scoring(results, accuracy_weight=0.5, removal_weight=0.5, percentage_weight=0.3):
    best_score = 0
    best_policy = None

    for result in results:
        acc, removed_nodes, percent_removed = result
        score = (accuracy_weight * acc) + (removal_weight * removed_nodes) + (percentage_weight * percent_removed)

        if score > best_score:
            best_score = score
            best_policy = result

    print(f"\nOptimal Policy by Weighted Scoring: Accuracy = {best_policy[0]:.4f}, Backdoor Nodes Removed = {best_policy[1]}, Weighted Score = {best_score:.4f}")
    return best_policy

# Pareto Efficiency
def pareto_optimal_selection(results):
    pareto_optimal = []

    for i, result in enumerate(results):
        dominated = False
        for j, other in enumerate(results):
            if i != j and all([other[k] >= result[k] for k in range(3)]) and any([other[k] > result[k] for k in range(3)]):
                dominated = True
                break
        if not dominated:
            pareto_optimal.append(result)

    print(f"\nPareto-optimal Policies: {len(pareto_optimal)}")
    for policy in pareto_optimal:
        print(f"Accuracy: {policy[0]:.4f}, Backdoor Nodes Removed: {policy[1]}, Percentage Removed: {policy[2]:.2f}%")

    return pareto_optimal

# Cost-Benefit Analysis
def cost_benefit_analysis(results, costs):
    if len(costs) != len(results):
        print(f"Error: Mismatch between number of results ({len(results)}) and costs ({len(costs)}).")
        return None

    best_benefit_cost_ratio = 0
    best_policy = None

    for i, result in enumerate(results):
        acc, removed_nodes, percent_removed = result
        cost = costs[i]

        benefit_cost_ratio = acc / cost

        if benefit_cost_ratio > best_benefit_cost_ratio:
            best_benefit_cost_ratio = benefit_cost_ratio
            best_policy = result

    print(f"\nOptimal Policy by Cost-Benefit Analysis: Accuracy = {best_policy[0]:.4f}, Backdoor Nodes Removed = {best_policy[1]}, Benefit-Cost Ratio = {best_benefit_cost_ratio:.4f}")
    return best_policy

# Robustness Check
def robustness_check(results, n_runs=5):
    averaged_results = []

    for result in results:
        total_acc, total_removed_nodes, total_percent_removed = 0, 0, 0
        for _ in range(n_runs):
            acc, removed_nodes, percent_removed = result  # You'd need to re-run the experiments here in a real scenario
            total_acc += acc
            total_removed_nodes += removed_nodes
            total_percent_removed += percent_removed

        avg_acc = total_acc / n_runs
        avg_removed_nodes = total_removed_nodes / n_runs
        avg_percent_removed = total_percent_removed / n_runs
        averaged_results.append((avg_acc, avg_removed_nodes, avg_percent_removed))

    best_result = max(averaged_results, key=lambda x: x[0])  # Select based on accuracy
    print(f"\nOptimal Policy after Robustness Check: Accuracy = {best_result[0]:.4f}, Backdoor Nodes Removed = {best_result[1]}, Avg Percentage Removed: {best_result[2]:.2f}%")
    return best_result

# Multi-Objective Optimization
def multi_objective_optimization(results, accuracy_weight=0.6, removal_weight=0.4):
    pareto_optimal = pareto_optimal_selection(results)

    best_score = 0
    best_policy = None

    for result in pareto_optimal:
        acc, removed_nodes, _ = result
        score = (accuracy_weight * acc) + (removal_weight * removed_nodes)

        if score > best_score:
            best_score = score
            best_policy = result

    print(f"\nMulti-objective Optimal Policy: Accuracy = {best_policy[0]:.4f}, Backdoor Nodes Removed = {best_policy[1]}, Weighted Score = {best_score:.4f}")
    return best_policy

# Running all attack, augmentation, and filter combinations
attacks = [(inject_feature_attack, "Feature-based Attack"),
           (inject_edge_attack, "Edge-based Attack"),
           (inject_trigger_attack, "Trigger-based Attack")]

augmentations = [(add_node_feature_noise, "Node Feature Noise"),
                 (random_edge_dropout, "Random Edge Dropout")]

filters = [(community_detection_filter, "Community Detection"),
           (pca_embedding_filter, "PCA Embedding"),
           (spectral_analysis_filter, "Spectral Analysis")]

# First, show the clean accuracy
clean_accuracy = run_clean_model()

# Running the new attack, augmentation, and filter combinations and gathering results
results = []
for attack_func, attack_name in attacks:
    for augment_func, augment_name in augmentations:
        for filter_func, filter_name in filters:
            result = run_case(attack_func, augment_func, filter_func, attack_name, augment_name, filter_name, num_poisoned=5)
            results.append(result)

# Example Costs for Cost-Benefit Analysis
costs = [1, 2, 3, 1, 2, 3, 1, 2, 3]  # Make sure this matches the number of attack + augment + filter combinations

# Select and display the optimal policy using different techniques
optimal_weighted = weighted_scoring(results)
optimal_pareto = pareto_optimal_selection(results)
optimal_cost_benefit = cost_benefit_analysis(results, costs)
optimal_robustness = robustness_check(results)
optimal_multi_objective = multi_objective_optimization(results)


Training the model on clean data...
Clean Accuracy: 0.7940

===== Feature-based Attack, Node Feature Noise, and Community Detection Filtration =====
Training the model on poisoned and augmented data (Feature-based Attack + Node Feature Noise)...
Accuracy on poisoned data (Feature-based Attack + Node Feature Noise): 0.7890
Backdoor Nodes Identified: 0/5
Percentage of Backdoor Nodes Removed: 0.00%
Retraining the model after Community Detection filtering...
Accuracy after Community Detection filter (Feature-based Attack + Node Feature Noise): 0.7830

===== Feature-based Attack, Node Feature Noise, and PCA Embedding Filtration =====
Training the model on poisoned and augmented data (Feature-based Attack + Node Feature Noise)...
Accuracy on poisoned data (Feature-based Attack + Node Feature Noise): 0.7890
Backdoor Nodes Identified: 5/5
Percentage of Backdoor Nodes Removed: 100.00%
Retraining the model after PCA Embedding filtering...
Accuracy after PCA Embedding filter (Feature-based Attack

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import to_networkx, remove_self_loops
import random
import networkx as nx
from sklearn.decomposition import PCA
import numpy as np

# Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')

# Define a simple GCN model
class GCN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 16)
        self.conv2 = GCNConv(16, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Load data
original_data = dataset[0]  # Save the original dataset separately

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the GCN model
def train(model, data):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    model.train()

    for epoch in range(200):
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

# Evaluate the model
def test(model, data):
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    acc = int(correct) / int(data.test_mask.sum())
    return acc

# Backdoor Attack 1: Feature-based Poisoning
def inject_feature_attack(data, num_poisoned_nodes=5):
    poisoned_nodes = random.sample(range(data.num_nodes), num_poisoned_nodes)
    for node in poisoned_nodes:
        data.x[node] = torch.rand_like(data.x[node])  # Randomize features as a trigger
        data.y[node] = random.randint(0, dataset.num_classes - 1)  # Mislabel
    return data, poisoned_nodes

# Backdoor Attack 2: Edge-based Poisoning
def inject_edge_attack(data, num_poisoned_edges=10):
    edge_index, _ = remove_self_loops(data.edge_index)
    poisoned_edges = []
    for _ in range(num_poisoned_edges):
        node1 = random.randint(0, data.num_nodes - 1)
        node2 = random.randint(0, data.num_nodes - 1)
        poisoned_edges.append([node1, node2])
    poisoned_edges = torch.tensor(poisoned_edges, dtype=torch.long).t().contiguous()
    data.edge_index = torch.cat([edge_index, poisoned_edges], dim=1)
    return data, poisoned_edges

# Backdoor Attack 3: Trigger-based Poisoning
def inject_trigger_attack(data, num_trigger_nodes=5, trigger_pattern=None):
    poisoned_nodes = random.sample(range(data.num_nodes), num_trigger_nodes)
    if trigger_pattern is None:
        trigger_pattern = torch.ones(data.num_node_features)

    for node in poisoned_nodes:
        data.x[node] = trigger_pattern  # Set a specific trigger pattern in features
        data.y[node] = random.randint(0, dataset.num_classes - 1)  # Mislabel

    return data, poisoned_nodes

# Augmentation Method 1: Node Feature Noise
def add_node_feature_noise(data, noise_level=0.1):
    noise = torch.randn_like(data.x) * noise_level
    data.x += noise  # Adding noise to node features
    return data

# Augmentation Method 2: Random Edge Dropout
def random_edge_dropout(data, drop_prob=0.1):
    edge_index, _ = remove_self_loops(data.edge_index)
    num_edges = edge_index.size(1)
    drop_mask = torch.rand(num_edges) > drop_prob  # Drop edges with probability drop_prob
    data.edge_index = edge_index[:, drop_mask]
    return data

# Filtration Method 1: Community Detection-Based Filtration
def community_detection_filter(data):
    G = to_networkx(data, to_undirected=True)
    communities = nx.algorithms.community.greedy_modularity_communities(G)
    filtered_nodes = []

    # Assuming smaller communities are potentially poisoned
    for community in communities:
        if len(community) < 10:  # Arbitrary threshold
            filtered_nodes.extend(list(community))

    return filtered_nodes

# Filtration Method 2: PCA Embedding-Based Detection
def pca_embedding_filter(data, n_components=2):
    X = data.x.cpu().detach().numpy()
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X)

    # Compute distances from the mean of embeddings
    distances = np.linalg.norm(X_pca - X_pca.mean(axis=0), axis=1)
    threshold = np.mean(distances) + 2 * np.std(distances)  # Arbitrary threshold
    filtered_nodes = np.where(distances > threshold)[0]

    return filtered_nodes

# Filtration Method 3: Spectral Analysis
def spectral_analysis_filter(data):
    G = to_networkx(data, to_undirected=True)
    laplacian_matrix = nx.laplacian_matrix(G).toarray()
    eigenvalues = np.linalg.eigvals(laplacian_matrix)

    # Identify anomalies by analyzing the spectrum of eigenvalues
    threshold = np.mean(eigenvalues) + 2 * np.std(eigenvalues)  # Arbitrary threshold
    filtered_nodes = [i for i, e in enumerate(eigenvalues) if e > threshold]

    return filtered_nodes

# Final Retraining after filtering poisoned nodes
def remove_filtered_nodes(data, filtered_nodes):
    data.train_mask[filtered_nodes] = False  # Remove poisoned nodes from training
    return data

# Train on clean data
def run_clean_model():
    print("Training the model on clean data...")
    data = original_data.clone().to(device)  # Reset to original clean data
    model = GCN(dataset.num_node_features, dataset.num_classes).to(device)
    train(model, data)
    acc_clean = test(model, data)
    print(f"Clean Accuracy: {acc_clean:.4f}")
    return acc_clean

# Function to execute one case with augmentation
def run_case(attack_func, augment_func, filter_func, attack_name, augment_name, filter_name, num_poisoned, acc_clean):
    print(f"\n===== {attack_name}, {augment_name}, and {filter_name} Filtration =====")

    # Step 1: Reset to original clean data before attack
    data = original_data.clone().to(device)
    model = GCN(dataset.num_node_features, dataset.num_classes).to(device)

    # Step 2: Apply Backdoor Attack
    data, poisoned_nodes = attack_func(data, num_poisoned)

    # Step 3: Apply Augmentation
    data = augment_func(data)

    print(f"Training the model on poisoned and augmented data ({attack_name} + {augment_name})...")
    train(model, data)
    acc_poisoned = test(model, data)
    print(f"Accuracy on poisoned data ({attack_name} + {augment_name}): {acc_poisoned:.4f}")

    # Calculate Attack Success Rate (ASR)
    attack_success_rate = 1 - (acc_poisoned / acc_clean)
    print(f"Attack Success Rate: {attack_success_rate:.4f}")

    # Step 4: Apply Filtration
    filtered_nodes = filter_func(data)
    num_identified_backdoor = len(set(filtered_nodes) & set(poisoned_nodes))
    percent_removed = (num_identified_backdoor / len(poisoned_nodes)) * 100

    print(f"Backdoor Nodes Identified: {num_identified_backdoor}/{len(poisoned_nodes)}")
    print(f"Percentage of Backdoor Nodes Removed: {percent_removed:.2f}%")

    data = remove_filtered_nodes(data, filtered_nodes)

    print(f"Retraining the model after {filter_name} filtering...")
    train(model, data)
    acc_defended = test(model, data)
    print(f"Accuracy after {filter_name} filter ({attack_name} + {augment_name}): {acc_defended:.4f}")

    return acc_defended, num_identified_backdoor, percent_removed, attack_success_rate

# Weighted Scoring Method
def weighted_scoring(results, accuracy_weight=0.5, removal_weight=0.5, percentage_weight=0.3, asr_weight=0.2):
    best_score = 0
    best_policy = None

    for result in results:
        acc, removed_nodes, percent_removed, asr = result
        score = (accuracy_weight * acc) + (removal_weight * removed_nodes) + (percentage_weight * percent_removed) + (asr_weight * asr)

        if score > best_score:
            best_score = score
            best_policy = result

    print(f"\nOptimal Policy by Weighted Scoring: Accuracy = {best_policy[0]:.4f}, Backdoor Nodes Removed = {best_policy[1]}, Weighted Score = {best_score:.4f}")
    return best_policy

# Running all attack, augmentation, and filter combinations
attacks = [(inject_feature_attack, "Feature-based Attack"),
           (inject_edge_attack, "Edge-based Attack"),
           (inject_trigger_attack, "Trigger-based Attack")]

augmentations = [(add_node_feature_noise, "Node Feature Noise"),
                 (random_edge_dropout, "Random Edge Dropout")]

filters = [(community_detection_filter, "Community Detection"),
           (pca_embedding_filter, "PCA Embedding"),
           (spectral_analysis_filter, "Spectral Analysis")]

# First, show the clean accuracy
clean_accuracy = run_clean_model()

# Running the new attack, augmentation, and filter combinations and gathering results
results = []
for attack_func, attack_name in attacks:
    for augment_func, augment_name in augmentations:
        for filter_func, filter_name in filters:
            result = run_case(attack_func, augment_func, filter_func, attack_name, augment_name, filter_name, num_poisoned=5, acc_clean=clean_accuracy)
            results.append(result)

# Example of Weighted Scoring
optimal_weighted = weighted_scoring(results)


Training the model on clean data...
Clean Accuracy: 0.8040

===== Feature-based Attack, Node Feature Noise, and Community Detection Filtration =====
Training the model on poisoned and augmented data (Feature-based Attack + Node Feature Noise)...
Accuracy on poisoned data (Feature-based Attack + Node Feature Noise): 0.7720
Attack Success Rate: 0.0398
Backdoor Nodes Identified: 0/5
Percentage of Backdoor Nodes Removed: 0.00%
Retraining the model after Community Detection filtering...
Accuracy after Community Detection filter (Feature-based Attack + Node Feature Noise): 0.7730

===== Feature-based Attack, Node Feature Noise, and PCA Embedding Filtration =====
Training the model on poisoned and augmented data (Feature-based Attack + Node Feature Noise)...
Accuracy on poisoned data (Feature-based Attack + Node Feature Noise): 0.7920
Attack Success Rate: 0.0149
Backdoor Nodes Identified: 5/5
Percentage of Backdoor Nodes Removed: 100.00%
Retraining the model after PCA Embedding filtering...
A

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
import copy
import numpy as np

# Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')

# Define the GCN model
class GCN(torch.nn.Module):
    def __init__(self, input_features, hidden_dim, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return x

# Function to train the model
def train(model, data, optimizer):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

# Function to test the model
def test(model, data):
    model.eval()
    logits = model(data)
    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        pred = logits[mask].argmax(dim=1)
        acc = (pred == data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

# Prepare data
data = dataset[0]
input_features = dataset.num_node_features
hidden_dim = 16
num_classes = dataset.num_classes

# Train on clean data
clean_model = GCN(input_features, hidden_dim, num_classes)
optimizer = torch.optim.Adam(clean_model.parameters(), lr=0.01, weight_decay=5e-4)

for epoch in range(200):
    train(clean_model, data, optimizer)

clean_train_acc, clean_val_acc, clean_test_acc = test(clean_model, data)
print(f'Clean Test Accuracy: {clean_test_acc:.4f}')

# Helper function to inject backdoor
def inject_subgraph_trigger(data, trigger_nodes, target_class):
    poisoned_data = copy.deepcopy(data)
    num_nodes = data.num_nodes

    # Add edges to form a triangular subgraph (trigger)
    poisoned_data.edge_index = torch.cat([
        poisoned_data.edge_index,
        torch.tensor([[trigger_nodes[0], trigger_nodes[1]],
                      [trigger_nodes[1], trigger_nodes[2]],
                      [trigger_nodes[2], trigger_nodes[0]]], dtype=torch.long).t()
    ], dim=1)

    # Assign the target class to the poisoned nodes
    poisoned_data.y[trigger_nodes] = target_class
    return poisoned_data

def inject_feature_trigger(data, trigger_nodes, feature_dim, trigger_value, target_class):
    poisoned_data = copy.deepcopy(data)
    # Modify features to include trigger
    poisoned_data.x[trigger_nodes, feature_dim] = trigger_value
    # Assign the target class to the poisoned nodes
    poisoned_data.y[trigger_nodes] = target_class
    return poisoned_data

# Attack 1: Subgraph-Based Backdoor Attack
trigger_nodes = [0, 1, 2]  # Nodes to modify
target_class = data.y.max().item()  # Use the maximum class label as the target

poisoned_data_subgraph = inject_subgraph_trigger(data, trigger_nodes, target_class)

# Retrain the model on poisoned data
model_subgraph = GCN(input_features, hidden_dim, num_classes)
optimizer = torch.optim.Adam(model_subgraph.parameters(), lr=0.01, weight_decay=5e-4)

for epoch in range(200):
    train(model_subgraph, poisoned_data_subgraph, optimizer)

subgraph_train_acc, subgraph_val_acc, subgraph_test_acc = test(model_subgraph, poisoned_data_subgraph)
print(f'Subgraph Attack Test Accuracy: {subgraph_test_acc:.4f}')
print(f'Accuracy Drop: {clean_test_acc - subgraph_test_acc:.4f}')

# Attack 2: Feature-Based Backdoor Attack
feature_dim = 0  # Feature dimension to modify
trigger_value = data.x[:, feature_dim].max() + 10  # Set a high value as trigger

poisoned_data_feature = inject_feature_trigger(data, trigger_nodes, feature_dim, trigger_value, target_class)

# Retrain the model on poisoned data
model_feature = GCN(input_features, hidden_dim, num_classes)
optimizer = torch.optim.Adam(model_feature.parameters(), lr=0.01, weight_decay=5e-4)

for epoch in range(200):
    train(model_feature, poisoned_data_feature, optimizer)

feature_train_acc, feature_val_acc, feature_test_acc = test(model_feature, poisoned_data_feature)
print(f'Feature Attack Test Accuracy: {feature_test_acc:.4f}')
print(f'Accuracy Drop: {clean_test_acc - feature_test_acc:.4f}')


Processing...
Done!


Clean Test Accuracy: 0.7870
Subgraph Attack Test Accuracy: 0.7930
Accuracy Drop: -0.0060
Feature Attack Test Accuracy: 0.7900
Accuracy Drop: -0.0030


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
import copy
import numpy as np

# Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')

# Define the GCN model
class GCN(torch.nn.Module):
    def __init__(self, input_features, hidden_dim, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return x

# Function to train the model
def train(model, data, optimizer):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

# Function to test the model
def test(model, data):
    model.eval()
    logits = model(data)
    pred = logits.argmax(dim=1)
    correct = pred[data.test_mask] == data.y[data.test_mask]
    acc = correct.sum().item() / data.test_mask.sum().item()
    return acc

# Prepare data
data = dataset[0]
input_features = dataset.num_node_features
hidden_dim = 16
num_classes = dataset.num_classes

# Train on clean data
clean_model = GCN(input_features, hidden_dim, num_classes)
optimizer = torch.optim.Adam(clean_model.parameters(), lr=0.01, weight_decay=5e-4)

for epoch in range(200):
    train(clean_model, data, optimizer)

clean_test_acc = test(clean_model, data)
print(f'Clean Test Accuracy: {clean_test_acc:.4f}')

# Helper functions to inject backdoor attacks

# Attack 1: Subgraph-Based Backdoor Attack
def subgraph_backdoor_attack(data, trigger_nodes, target_class):
    poisoned_data = copy.deepcopy(data)
    # Add edges to form a triangle (subgraph trigger)
    new_edges = torch.tensor([
        [trigger_nodes[0], trigger_nodes[1]],
        [trigger_nodes[1], trigger_nodes[2]],
        [trigger_nodes[2], trigger_nodes[0]]
    ], dtype=torch.long).t()
    poisoned_data.edge_index = torch.cat([poisoned_data.edge_index, new_edges], dim=1)
    # Assign target class to trigger nodes
    poisoned_data.y[trigger_nodes] = target_class
    return poisoned_data

# Attack 2: Feature-Based Backdoor Attack
def feature_backdoor_attack(data, trigger_nodes, feature_dim, trigger_value, target_class):
    poisoned_data = copy.deepcopy(data)
    # Modify features to include trigger
    poisoned_data.x[trigger_nodes, feature_dim] = trigger_value
    # Assign target class to trigger nodes
    poisoned_data.y[trigger_nodes] = target_class
    return poisoned_data

# Attack 3: Edge Manipulation Backdoor Attack
def edge_manipulation_backdoor_attack(data, trigger_node, connect_nodes, target_class):
    poisoned_data = copy.deepcopy(data)
    # Connect trigger_node to a set of nodes to form a pattern
    new_edges = torch.tensor([[trigger_node]*len(connect_nodes), connect_nodes], dtype=torch.long)
    poisoned_data.edge_index = torch.cat([poisoned_data.edge_index, new_edges], dim=1)
    # Assign target class to trigger node
    poisoned_data.y[trigger_node] = target_class
    return poisoned_data

# Attack 4: Global Trigger Backdoor Attack
def global_trigger_backdoor_attack(data, scaling_factor, target_class):
    poisoned_data = copy.deepcopy(data)
    # Scale all node features by scaling_factor
    poisoned_data.x = poisoned_data.x * scaling_factor
    # Assign target class to all nodes (extreme case)
    poisoned_data.y[:] = target_class
    return poisoned_data

# Attack 5: Local Trigger Backdoor Attack
def local_trigger_backdoor_attack(data, trigger_node, neighbor_nodes, target_class):
    poisoned_data = copy.deepcopy(data)
    # Modify a subgraph around trigger_node
    for neighbor in neighbor_nodes:
        poisoned_data.x[neighbor] = poisoned_data.x[neighbor] * 1.5
    # Assign target class to trigger node
    poisoned_data.y[trigger_node] = target_class
    return poisoned_data

# Function to retrain and test the model after an attack
def retrain_and_evaluate(attack_name, poisoned_data):
    model = GCN(input_features, hidden_dim, num_classes)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    for epoch in range(200):
        train(model, poisoned_data, optimizer)
    test_acc = test(model, poisoned_data)
    accuracy_drop = clean_test_acc - test_acc
    print(f'{attack_name} Test Accuracy: {test_acc:.4f}')
    print(f'Accuracy Drop: {accuracy_drop:.4f}\n')

# Apply Attack 1: Subgraph-Based Backdoor Attack
trigger_nodes = [0, 1, 2]  # Nodes to modify
target_class = data.y.max().item()  # Use the maximum class label as the target
poisoned_data = subgraph_backdoor_attack(data, trigger_nodes, target_class)
print("Attack 1: Subgraph-Based Backdoor Attack")
retrain_and_evaluate("Subgraph Attack", poisoned_data)

# Apply Attack 2: Feature-Based Backdoor Attack
feature_dim = 0  # Feature dimension to modify
trigger_value = data.x[:, feature_dim].max() + 10  # Set a high value as trigger
poisoned_data = feature_backdoor_attack(data, trigger_nodes, feature_dim, trigger_value, target_class)
print("Attack 2: Feature-Based Backdoor Attack")
retrain_and_evaluate("Feature Attack", poisoned_data)

# Apply Attack 3: Edge Manipulation Backdoor Attack
trigger_node = 0
connect_nodes = [10, 20, 30]  # Nodes to connect with trigger_node
poisoned_data = edge_manipulation_backdoor_attack(data, trigger_node, connect_nodes, target_class)
print("Attack 3: Edge Manipulation Backdoor Attack")
retrain_and_evaluate("Edge Manipulation Attack", poisoned_data)

# Apply Attack 4: Global Trigger Backdoor Attack
scaling_factor = 1.5  # Scaling factor for all node features
poisoned_data = global_trigger_backdoor_attack(data, scaling_factor, target_class)
print("Attack 4: Global Trigger Backdoor Attack")
retrain_and_evaluate("Global Trigger Attack", poisoned_data)

# Apply Attack 5: Local Trigger Backdoor Attack
trigger_node = 0
neighbor_nodes = data.edge_index[1][data.edge_index[0] == trigger_node].tolist()
poisoned_data = local_trigger_backdoor_attack(data, trigger_node, neighbor_nodes, target_class)
print("Attack 5: Local Trigger Backdoor Attack")
retrain_and_evaluate("Local Trigger Attack", poisoned_data)

# For Attacks 6 to 10, we will provide code snippets and explanations.

# Attack 6: Label-Consistent Backdoor Attack
# Description: Poisoned samples retain their original labels.

def label_consistent_backdoor_attack(data, trigger_nodes, feature_dim, trigger_value):
    poisoned_data = copy.deepcopy(data)
    poisoned_data.x[trigger_nodes, feature_dim] = trigger_value
    # Labels remain the same
    return poisoned_data

# Apply Attack 6
feature_dim = 0
trigger_value = data.x[:, feature_dim].mean()
poisoned_data = label_consistent_backdoor_attack(data, trigger_nodes, feature_dim, trigger_value)
print("Attack 6: Label-Consistent Backdoor Attack")
retrain_and_evaluate("Label-Consistent Attack", poisoned_data)

# Attack 7: Clean-Label Backdoor Attack
# Similar to Label-Consistent but focuses on making poisoned samples indistinguishable.

def clean_label_backdoor_attack(data, trigger_nodes, feature_dim, trigger_value):
    poisoned_data = copy.deepcopy(data)
    poisoned_data.x[trigger_nodes, feature_dim] = trigger_value
    # Labels remain the same
    return poisoned_data

# Apply Attack 7
poisoned_data = clean_label_backdoor_attack(data, trigger_nodes, feature_dim, trigger_value)
print("Attack 7: Clean-Label Backdoor Attack")
retrain_and_evaluate("Clean-Label Attack", poisoned_data)

# Attack 8: Dynamic Graph Backdoor Attack
# Not applicable to static datasets like Cora. Skipping implementation.

# Attack 9: Topological Backdoor Attack
def topological_backdoor_attack(data, trigger_nodes, target_class):
    poisoned_data = copy.deepcopy(data)
    # Create a unique community structure
    for i in range(len(trigger_nodes) - 1):
        poisoned_data.edge_index = torch.cat([
            poisoned_data.edge_index,
            torch.tensor([[trigger_nodes[i], trigger_nodes[i+1]]], dtype=torch.long).t()
        ], dim=1)
    # Assign target class
    poisoned_data.y[trigger_nodes] = target_class
    return poisoned_data

# Apply Attack 9
poisoned_data = topological_backdoor_attack(data, trigger_nodes, target_class)
print("Attack 9: Topological Backdoor Attack")
retrain_and_evaluate("Topological Attack", poisoned_data)

# Attack 10: Structural Backdoor Attack
def structural_backdoor_attack(data, trigger_node, target_degree, target_class):
    poisoned_data = copy.deepcopy(data)
    current_degree = (poisoned_data.edge_index[0] == trigger_node).sum().item()
    # Adjust degree to target_degree
    while current_degree < target_degree:
        new_node = np.random.randint(0, data.num_nodes)
        poisoned_data.edge_index = torch.cat([
            poisoned_data.edge_index,
            torch.tensor([[trigger_node, new_node]], dtype=torch.long).t()
        ], dim=1)
        current_degree += 1
    # Assign target class
    poisoned_data.y[trigger_node] = target_class
    return poisoned_data

# Apply Attack 10
target_degree = 50  # Increase degree to 50
poisoned_data = structural_backdoor_attack(data, trigger_node, target_degree, target_class)
print("Attack 10: Structural Backdoor Attack")
retrain_and_evaluate("Structural Attack", poisoned_data)


Clean Test Accuracy: 0.8070
Attack 1: Subgraph-Based Backdoor Attack
Subgraph Attack Test Accuracy: 0.7910
Accuracy Drop: 0.0160

Attack 2: Feature-Based Backdoor Attack
Feature Attack Test Accuracy: 0.8040
Accuracy Drop: 0.0030

Attack 3: Edge Manipulation Backdoor Attack
Edge Manipulation Attack Test Accuracy: 0.7880
Accuracy Drop: 0.0190

Attack 4: Global Trigger Backdoor Attack
Global Trigger Attack Test Accuracy: 1.0000
Accuracy Drop: -0.1930

Attack 5: Local Trigger Backdoor Attack
Local Trigger Attack Test Accuracy: 0.7840
Accuracy Drop: 0.0230

Attack 6: Label-Consistent Backdoor Attack
Label-Consistent Attack Test Accuracy: 0.8040
Accuracy Drop: 0.0030

Attack 7: Clean-Label Backdoor Attack
Clean-Label Attack Test Accuracy: 0.8040
Accuracy Drop: 0.0030

Attack 9: Topological Backdoor Attack
Topological Attack Test Accuracy: 0.7830
Accuracy Drop: 0.0240

Attack 10: Structural Backdoor Attack
Structural Attack Test Accuracy: 0.7810
Accuracy Drop: 0.0260



In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import to_networkx, from_networkx
import copy
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from scipy.stats import zscore

# Load the Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')

# Define the GCN model
class GCN(torch.nn.Module):
    def __init__(self, input_features, hidden_dim, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return x

# Function to train the model
def train(model, data, optimizer):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

# Function to test the model
def test(model, data):
    model.eval()
    logits = model(data)
    pred = logits.argmax(dim=1)
    correct = pred[data.test_mask] == data.y[data.test_mask]
    acc = correct.sum().item() / data.test_mask.sum().item()
    return acc

# Prepare data
data = dataset[0]
input_features = dataset.num_node_features
hidden_dim = 16
num_classes = dataset.num_classes

# Train on clean data
clean_model = GCN(input_features, hidden_dim, num_classes)
optimizer = torch.optim.Adam(clean_model.parameters(), lr=0.01, weight_decay=5e-4)

for epoch in range(200):
    train(clean_model, data, optimizer)

clean_test_acc = test(clean_model, data)
print(f'Clean Test Accuracy: {clean_test_acc:.4f}\n')

# Helper functions to inject backdoor attacks

# Attack 1: Subgraph-Based Backdoor Attack
def subgraph_backdoor_attack(data, trigger_nodes, target_class):
    poisoned_data = copy.deepcopy(data)
    # Add edges to form a triangle (subgraph trigger)
    new_edges = torch.tensor([
        [trigger_nodes[0], trigger_nodes[1]],
        [trigger_nodes[1], trigger_nodes[2]],
        [trigger_nodes[2], trigger_nodes[0]]
    ], dtype=torch.long).t()
    poisoned_data.edge_index = torch.cat([poisoned_data.edge_index, new_edges], dim=1)
    # Assign target class to trigger nodes
    poisoned_data.y[trigger_nodes] = target_class
    return poisoned_data

# Defense Method: Anomaly Detection using Z-Score on Degrees
def filter_anomalies_by_degree(data):
    # Convert edge_index to adjacency matrix
    adj = torch.zeros((data.num_nodes, data.num_nodes))
    adj[data.edge_index[0], data.edge_index[1]] = 1
    degrees = adj.sum(dim=1).numpy()
    # Compute Z-scores
    z_scores = zscore(degrees)
    # Nodes with absolute Z-score > threshold are considered anomalies
    threshold = 3  # Adjust as needed
    normal_nodes = np.where(np.abs(z_scores) <= threshold)[0]
    # Filter nodes and edges
    filtered_data = copy.deepcopy(data)
    mask = torch.zeros(data.num_nodes, dtype=torch.bool)
    mask[normal_nodes] = True
    filtered_data.x = data.x[mask]
    filtered_data.y = data.y[mask]
    node_idx = torch.arange(data.num_nodes)[mask]
    node_mapping = {old_idx.item(): new_idx for new_idx, old_idx in enumerate(node_idx)}
    edge_mask = mask[data.edge_index[0]] & mask[data.edge_index[1]]
    filtered_data.edge_index = data.edge_index[:, edge_mask]
    # Re-map node indices
    filtered_data.edge_index = torch.tensor([
        [node_mapping[idx.item()] for idx in filtered_data.edge_index[0]],
        [node_mapping[idx.item()] for idx in filtered_data.edge_index[1]]
    ], dtype=torch.long)
    filtered_data.train_mask = data.train_mask[mask]
    filtered_data.val_mask = data.val_mask[mask]
    filtered_data.test_mask = data.test_mask[mask]
    return filtered_data

# Function to retrain and evaluate the model after applying defense
def retrain_and_evaluate_defense(attack_name, poisoned_data, defense_method=None):
    if defense_method:
        defended_data = defense_method(poisoned_data)
    else:
        defended_data = poisoned_data
    model = GCN(input_features, hidden_dim, num_classes)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    for epoch in range(200):
        train(model, defended_data, optimizer)
    test_acc = test(model, defended_data)
    accuracy_drop = clean_test_acc - test_acc
    print(f'{attack_name} Test Accuracy after Defense: {test_acc:.4f}')
    print(f'Accuracy Drop after Defense: {accuracy_drop:.4f}\n')
    return test_acc

# Apply Attack 1: Subgraph-Based Backdoor Attack
trigger_nodes = [0, 1, 2]  # Nodes to modify
target_class = data.y.max().item()  # Use the maximum class label as the target
poisoned_data = subgraph_backdoor_attack(data, trigger_nodes, target_class)
print("Attack 1: Subgraph-Based Backdoor Attack")
# Retrain without defense
model = GCN(input_features, hidden_dim, num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
for epoch in range(200):
    train(model, poisoned_data, optimizer)
test_acc = test(model, poisoned_data)
accuracy_drop = clean_test_acc - test_acc
print(f'Test Accuracy without Defense: {test_acc:.4f}')
print(f'Accuracy Drop without Defense: {accuracy_drop:.4f}')

# Apply Defense
print("Applying Defense: Anomaly Detection by Degree")
defended_test_acc = retrain_and_evaluate_defense("Subgraph Attack", poisoned_data, filter_anomalies_by_degree)

# Attack 2: Feature-Based Backdoor Attack
def feature_backdoor_attack(data, trigger_nodes, feature_dim, trigger_value, target_class):
    poisoned_data = copy.deepcopy(data)
    # Modify features to include trigger
    poisoned_data.x[trigger_nodes, feature_dim] = trigger_value
    # Assign target class to trigger nodes
    poisoned_data.y[trigger_nodes] = target_class
    return poisoned_data

# Defense Method: Anomaly Detection on Features using PCA
def filter_anomalies_by_pca(data):
    x_numpy = data.x.numpy()
    pca = PCA(n_components=2)
    x_pca = pca.fit_transform(x_numpy)
    # Perform clustering
    kmeans = KMeans(n_clusters=num_classes, random_state=0).fit(x_pca)
    distances = kmeans.transform(x_pca).min(axis=1)
    threshold = np.percentile(distances, 95)  # Adjust as needed
    normal_nodes = np.where(distances <= threshold)[0]
    # Filter nodes and edges
    filtered_data = copy.deepcopy(data)
    mask = torch.zeros(data.num_nodes, dtype=torch.bool)
    mask[normal_nodes] = True
    filtered_data.x = data.x[mask]
    filtered_data.y = data.y[mask]
    node_idx = torch.arange(data.num_nodes)[mask]
    node_mapping = {old_idx.item(): new_idx for new_idx, old_idx in enumerate(node_idx)}
    edge_mask = mask[data.edge_index[0]] & mask[data.edge_index[1]]
    filtered_data.edge_index = data.edge_index[:, edge_mask]
    # Re-map node indices
    filtered_data.edge_index = torch.tensor([
        [node_mapping[idx.item()] for idx in filtered_data.edge_index[0]],
        [node_mapping[idx.item()] for idx in filtered_data.edge_index[1]]
    ], dtype=torch.long)
    filtered_data.train_mask = data.train_mask[mask]
    filtered_data.val_mask = data.val_mask[mask]
    filtered_data.test_mask = data.test_mask[mask]
    return filtered_data

# Apply Attack 2
feature_dim = 0  # Feature dimension to modify
trigger_value = data.x[:, feature_dim].max() + 10  # Set a high value as trigger
poisoned_data = feature_backdoor_attack(data, trigger_nodes, feature_dim, trigger_value, target_class)
print("Attack 2: Feature-Based Backdoor Attack")
# Retrain without defense
model = GCN(input_features, hidden_dim, num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
for epoch in range(200):
    train(model, poisoned_data, optimizer)
test_acc = test(model, poisoned_data)
accuracy_drop = clean_test_acc - test_acc
print(f'Test Accuracy without Defense: {test_acc:.4f}')
print(f'Accuracy Drop without Defense: {accuracy_drop:.4f}')

# Apply Defense
print("Applying Defense: Anomaly Detection by PCA")
defended_test_acc = retrain_and_evaluate_defense("Feature Attack", poisoned_data, filter_anomalies_by_pca)

# Attack 3: Edge Manipulation Backdoor Attack
def edge_manipulation_backdoor_attack(data, trigger_node, connect_nodes, target_class):
    poisoned_data = copy.deepcopy(data)
    # Connect trigger_node to a set of nodes to form a pattern
    new_edges = torch.tensor([[trigger_node]*len(connect_nodes), connect_nodes], dtype=torch.long)
    poisoned_data.edge_index = torch.cat([poisoned_data.edge_index, new_edges], dim=1)
    # Assign target class to trigger node
    poisoned_data.y[trigger_node] = target_class
    return poisoned_data

# Defense Method: Edge Removal based on Betweenness Centrality
def filter_edges_by_betweenness(data):
    G = to_networkx(data, to_undirected=True)
    edge_betweenness = nx.edge_betweenness_centrality(G)
    # Remove edges with high betweenness centrality
    threshold = np.percentile(list(edge_betweenness.values()), 95)  # Adjust as needed
    edges_to_remove = [edge for edge, centrality in edge_betweenness.items() if centrality > threshold]
    G.remove_edges_from(edges_to_remove)
    # Convert back to PyTorch Geometric data
    filtered_data = copy.deepcopy(data)
    filtered_data.edge_index = from_networkx(G).edge_index
    return filtered_data

import networkx as nx  # Import networkx for graph analysis

# Apply Attack 3
trigger_node = 0
connect_nodes = [10, 20, 30]  # Nodes to connect with trigger_node
poisoned_data = edge_manipulation_backdoor_attack(data, trigger_node, connect_nodes, target_class)
print("Attack 3: Edge Manipulation Backdoor Attack")
# Retrain without defense
model = GCN(input_features, hidden_dim, num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
for epoch in range(200):
    train(model, poisoned_data, optimizer)
test_acc = test(model, poisoned_data)
accuracy_drop = clean_test_acc - test_acc
print(f'Test Accuracy without Defense: {test_acc:.4f}')
print(f'Accuracy Drop without Defense: {accuracy_drop:.4f}')

# Apply Defense
print("Applying Defense: Edge Filtering by Betweenness Centrality")
defended_test_acc = retrain_and_evaluate_defense("Edge Manipulation Attack", poisoned_data, filter_edges_by_betweenness)

# Attack 4: Global Trigger Backdoor Attack
def global_trigger_backdoor_attack(data, scaling_factor, target_class):
    poisoned_data = copy.deepcopy(data)
    # Scale all node features by scaling_factor
    poisoned_data.x = poisoned_data.x * scaling_factor
    # Assign target class to all nodes (extreme case)
    poisoned_data.y[:] = target_class
    return poisoned_data

# Defense Method: Feature Normalization
def normalize_features(data):
    normalized_data = copy.deepcopy(data)
    normalized_data.x = F.normalize(data.x, p=2, dim=1)
    return normalized_data

# Apply Attack 4
scaling_factor = 1.5  # Scaling factor for all node features
poisoned_data = global_trigger_backdoor_attack(data, scaling_factor, target_class)
print("Attack 4: Global Trigger Backdoor Attack")
# Retrain without defense
model = GCN(input_features, hidden_dim, num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
for epoch in range(200):
    train(model, poisoned_data, optimizer)
test_acc = test(model, poisoned_data)
accuracy_drop = clean_test_acc - test_acc
print(f'Test Accuracy without Defense: {test_acc:.4f}')
print(f'Accuracy Drop without Defense: {accuracy_drop:.4f}')

# Apply Defense
print("Applying Defense: Feature Normalization")
defended_data = normalize_features(poisoned_data)
defended_test_acc = retrain_and_evaluate_defense("Global Trigger Attack", defended_data)

# Attack 5: Local Trigger Backdoor Attack
def local_trigger_backdoor_attack(data, trigger_node, neighbor_nodes, target_class):
    poisoned_data = copy.deepcopy(data)
    # Modify features of neighbor nodes
    for neighbor in neighbor_nodes:
        poisoned_data.x[neighbor] = poisoned_data.x[neighbor] * 1.5
    # Assign target class to trigger node
    poisoned_data.y[trigger_node] = target_class
    return poisoned_data

# Defense Method: Graph Purification by Smoothing
def graph_purification_smoothing(data):
    purified_data = copy.deepcopy(data)
    edge_index = data.edge_index
    x = data.x
    # Simple smoothing: each node feature is averaged with its neighbors
    for node in range(data.num_nodes):
        neighbors = edge_index[1][edge_index[0] == node]
        if len(neighbors) > 0:
            neighbor_features = x[neighbors]
            purified_data.x[node] = (x[node] + neighbor_features.mean(dim=0)) / 2
    return purified_data

# Apply Attack 5
trigger_node = 0
neighbor_nodes = data.edge_index[1][data.edge_index[0] == trigger_node].tolist()
poisoned_data = local_trigger_backdoor_attack(data, trigger_node, neighbor_nodes, target_class)
print("Attack 5: Local Trigger Backdoor Attack")
# Retrain without defense
model = GCN(input_features, hidden_dim, num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
for epoch in range(200):
    train(model, poisoned_data, optimizer)
test_acc = test(model, poisoned_data)
accuracy_drop = clean_test_acc - test_acc
print(f'Test Accuracy without Defense: {test_acc:.4f}')
print(f'Accuracy Drop without Defense: {accuracy_drop:.4f}')

# Apply Defense
print("Applying Defense: Graph Purification by Smoothing")
defended_data = graph_purification_smoothing(poisoned_data)
defended_test_acc = retrain_and_evaluate_defense("Local Trigger Attack", defended_data)


Clean Test Accuracy: 0.7970

Attack 1: Subgraph-Based Backdoor Attack
Test Accuracy without Defense: 0.7930
Accuracy Drop without Defense: 0.0040
Applying Defense: Anomaly Detection by Degree
Subgraph Attack Test Accuracy after Defense: 0.7759
Accuracy Drop after Defense: 0.0211

Attack 2: Feature-Based Backdoor Attack
Test Accuracy without Defense: 0.7950
Accuracy Drop without Defense: 0.0020
Applying Defense: Anomaly Detection by PCA
Feature Attack Test Accuracy after Defense: 0.7794
Accuracy Drop after Defense: 0.0176

Attack 3: Edge Manipulation Backdoor Attack
Test Accuracy without Defense: 0.7830
Accuracy Drop without Defense: 0.0140
Applying Defense: Edge Filtering by Betweenness Centrality
Edge Manipulation Attack Test Accuracy after Defense: 0.7860
Accuracy Drop after Defense: 0.0110

Attack 4: Global Trigger Backdoor Attack
Test Accuracy without Defense: 1.0000
Accuracy Drop without Defense: -0.2030
Applying Defense: Feature Normalization
Global Trigger Attack Test Accuracy a

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.transforms import NormalizeFeatures
from sklearn.metrics import accuracy_score

# Load and preprocess Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora', transform=NormalizeFeatures())
data = dataset[0]

# Define GNN model
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Initialize model, optimizer, and data
model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)
model = model.to(device)

# Function to train the model
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

# Function to test the model
def test():
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)
    acc = accuracy_score(data.y[data.test_mask].cpu(), pred[data.test_mask].cpu())
    return acc

# Train and evaluate on clean data
clean_accuracies = []
for epoch in range(200):
    train()
    acc = test()
    clean_accuracies.append(acc)
print("Accuracy on clean data:", clean_accuracies[-1])

# Implement Poisoning Attacks
# Here, create different poisoned versions of the dataset:
# Node-Level Attack: Modify node features.
# Edge-Level Attack: Add or remove edges.
# Subgraph Attack: Inject an adversarial subgraph.

def node_level_attack(data, poison_rate=0.5):
    num_poison = int(poison_rate * data.num_nodes)
    for _ in range(num_poison):
        node = torch.randint(0, data.num_nodes, (1,))
        data.x[node] = torch.rand(data.x[node].shape)
    return data

def edge_level_attack(data, poison_rate=0.5):
    num_poison = int(poison_rate * data.num_edges)
    edge_index = data.edge_index.clone()

    for _ in range(num_poison):
        node1 = torch.randint(0, data.num_nodes, (1,))
        node2 = torch.randint(0, data.num_nodes, (1,))
        new_edge = torch.tensor([[node1.item()], [node2.item()]])
        edge_index = torch.cat([edge_index, new_edge], dim=1)

    data.edge_index = edge_index
    return data

def subgraph_attack(data, poison_rate=0.5):
    edge_index = data.edge_index.clone()
    subgraph_size = int(poison_rate * data.num_nodes)

    for _ in range(subgraph_size):
        node1 = torch.randint(0, data.num_nodes, (1,))
        node2 = torch.randint(0, data.num_nodes, (1,))
        new_edge = torch.tensor([[node1.item()], [node2.item()]])
        edge_index = torch.cat([edge_index, new_edge], dim=1)

    data.edge_index = edge_index
    return data


# Apply poisoning and test
poisoned_accuracies = {}
for attack_fn in [node_level_attack, edge_level_attack, subgraph_attack]:
    poisoned_data = attack_fn(data.clone())
    poisoned_acc = []
    for epoch in range(200):
        train()
        poisoned_acc.append(test())
    poisoned_accuracies[attack_fn.__name__] = poisoned_acc
    print(f"Accuracy after {attack_fn.__name__}:", poisoned_acc[-1])

# Applying Augmentation (e.g., dropout, feature smoothing, etc.) to improve poisoned accuracy
def augment(data):
    data.x = F.dropout(data.x, p=0.5, training=True)  # Example of feature dropout
    # Additional augmentations can be added here.
    return data

# Retrain with augmented poisoned data
augmented_accuracies = {}
for attack_fn in [node_level_attack, edge_level_attack, subgraph_attack]:
    poisoned_data = augment(attack_fn(data.clone()))
    augmented_acc = []
    for epoch in range(200):
        train()
        augmented_acc.append(test())
    augmented_accuracies[attack_fn.__name__] = augmented_acc
    print(f"Accuracy after augmentation of {attack_fn.__name__}:", augmented_acc[-1])

# Display accuracy drops and improvements
print("Accuracy on clean data:", clean_accuracies[-1])
for attack, acc in poisoned_accuracies.items():
    print(f"Accuracy drop after {attack}: {clean_accuracies[-1] - acc[-1]}")
    print(f"Recovered accuracy after augmentation: {augmented_accuracies[attack][-1]}")


Accuracy on clean data: 0.82
Accuracy after node_level_attack: 0.813
Accuracy after edge_level_attack: 0.809
Accuracy after subgraph_attack: 0.813
Accuracy after augmentation of node_level_attack: 0.815
Accuracy after augmentation of edge_level_attack: 0.818
Accuracy after augmentation of subgraph_attack: 0.801
Accuracy on clean data: 0.82
Accuracy drop after node_level_attack: 0.007000000000000006
Recovered accuracy after augmentation: 0.815
Accuracy drop after edge_level_attack: 0.010999999999999899
Recovered accuracy after augmentation: 0.818
Accuracy drop after subgraph_attack: 0.007000000000000006
Recovered accuracy after augmentation: 0.801


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.transforms import NormalizeFeatures
from sklearn.metrics import accuracy_score

# Load and preprocess Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora', transform=NormalizeFeatures())
data = dataset[0]

# Define GNN model
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Initialize model, optimizer, and data
model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)
model = model.to(device)

# Function to train the model
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

# Function to test the model
def test():
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)
    acc = accuracy_score(data.y[data.test_mask].cpu(), pred[data.test_mask].cpu())
    return acc

# Train and evaluate on clean data
clean_accuracies = []
for epoch in range(200):
    train()
    acc = test()
    clean_accuracies.append(acc)
print("Accuracy on clean data:", clean_accuracies[-1])

# Poisoning Attack Functions
def node_level_attack(data, poison_rate=0.1):
    num_poison = int(poison_rate * data.num_nodes)
    for _ in range(num_poison):
        node = torch.randint(0, data.num_nodes, (1,))
        data.x[node] = torch.rand(data.x[node].shape)
    return data

def edge_level_attack(data, poison_rate=0.1):
    num_poison = int(poison_rate * data.num_edges)
    edge_index = data.edge_index.clone()

    for _ in range(num_poison):
        node1 = torch.randint(0, data.num_nodes, (1,))
        node2 = torch.randint(0, data.num_nodes, (1,))
        new_edge = torch.tensor([[node1.item()], [node2.item()]])
        edge_index = torch.cat([edge_index, new_edge], dim=1)

    data.edge_index = edge_index
    return data

def subgraph_attack(data, poison_rate=0.1):
    edge_index = data.edge_index.clone()
    subgraph_size = int(poison_rate * data.num_nodes)

    for _ in range(subgraph_size):
        node1 = torch.randint(0, data.num_nodes, (1,))
        node2 = torch.randint(0, data.num_nodes, (1,))
        new_edge = torch.tensor([[node1.item()], [node2.item()]])
        edge_index = torch.cat([edge_index, new_edge], dim=1)

    data.edge_index = edge_index
    return data

# Augmentation Functions
def node_level_augmentation(data):
    data.x = F.dropout(data.x, p=0.1, training=True)
    return data

def edge_level_augmentation(data):
    edge_index = data.edge_index
    mask = torch.rand(edge_index.size(1)) > 0.1
    data.edge_index = edge_index[:, mask]
    return data

def subgraph_augmentation(data, subgraph_nodes):
    edge_index = data.edge_index
    for node in subgraph_nodes:
        target = torch.randint(0, data.num_nodes, (1,))
        new_edge = torch.tensor([[node], [target]])
        edge_index = torch.cat([edge_index, new_edge], dim=1)
    data.edge_index = edge_index
    return data

def graph_level_augmentation(data):
    noise = torch.randn_like(data.x) * 0.1
    data.x = data.x + noise
    return data

# Apply Poisoning Attacks and test accuracy, then apply augmentations and re-test accuracy
results = {}

# Define subgraph nodes for subgraph augmentation
subgraph_nodes = torch.randperm(data.num_nodes)[:int(0.1 * data.num_nodes)]

# Run each attack, measure accuracy, then apply corresponding augmentation
for attack_fn, augment_fn, augment_name in [
    (node_level_attack, node_level_augmentation, "Node-Level Augmentation"),
    (edge_level_attack, edge_level_augmentation, "Edge-Level Augmentation"),
    (subgraph_attack, lambda d: subgraph_augmentation(d, subgraph_nodes), "Subgraph Augmentation"),
    (subgraph_attack, graph_level_augmentation, "Graph-Level Augmentation")  # Using subgraph attack for graph-level test
]:
    # Reset model to baseline and apply attack
    model = GCN().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    poisoned_data = attack_fn(data.clone())

    # Train on attacked data
    attack_acc = []
    for epoch in range(200):
        train()
        attack_acc.append(test())

    print(f"Accuracy after {attack_fn.__name__}:", attack_acc[-1])

    # Apply augmentation to the attacked data
    augmented_data = augment_fn(poisoned_data.clone())

    # Reset model and retrain on augmented data
    model = GCN().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    augmented_acc = []
    for epoch in range(200):
        train()
        augmented_acc.append(test())

    print(f"Accuracy after {augment_name}:", augmented_acc[-1])

    # Store results
    results[attack_fn.__name__] = {
        "poisoned_accuracy": attack_acc[-1],
        "augmented_accuracy": augmented_acc[-1]
    }

# Display results
print("\nFinal Results:")
for attack, result in results.items():
    print(f"{attack}:")
    print(f"  Accuracy after attack: {result['poisoned_accuracy']}")
    print(f"  Accuracy after augmentation: {result['augmented_accuracy']}")


Accuracy on clean data: 0.8
Accuracy after node_level_attack: 0.81
Accuracy after Node-Level Augmentation: 0.812
Accuracy after edge_level_attack: 0.8
Accuracy after Edge-Level Augmentation: 0.807
Accuracy after subgraph_attack: 0.812
Accuracy after Subgraph Augmentation: 0.813
Accuracy after subgraph_attack: 0.813
Accuracy after Graph-Level Augmentation: 0.807

Final Results:
node_level_attack:
  Accuracy after attack: 0.81
  Accuracy after augmentation: 0.812
edge_level_attack:
  Accuracy after attack: 0.8
  Accuracy after augmentation: 0.807
subgraph_attack:
  Accuracy after attack: 0.813
  Accuracy after augmentation: 0.807


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.transforms import NormalizeFeatures
from sklearn.metrics import accuracy_score

# Load and preprocess Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora', transform=NormalizeFeatures())
data = dataset[0]

# Define GNN model
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Initialize model, optimizer, and data
model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)
model = model.to(device)

# Function to train the model
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

# Function to test the model
def test():
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)
    acc = accuracy_score(data.y[data.test_mask].cpu(), pred[data.test_mask].cpu())
    return acc

# Train and evaluate on clean data
clean_accuracies = []
for epoch in range(200):
    train()
    acc = test()
    clean_accuracies.append(acc)
print("Accuracy on clean data:", clean_accuracies[-1])

# Poisoning Attack Functions
def node_level_attack(data, poison_rate=0.20):
    num_poison = int(poison_rate * data.num_nodes)
    for _ in range(num_poison):
        node = torch.randint(0, data.num_nodes, (1,))
        data.x[node] = torch.rand(data.x[node].shape)
    return data

def edge_level_attack(data, poison_rate=0.20):
    num_poison = int(poison_rate * data.num_edges)
    edge_index = data.edge_index.clone()

    for _ in range(num_poison):
        node1 = torch.randint(0, data.num_nodes, (1,))
        node2 = torch.randint(0, data.num_nodes, (1,))
        new_edge = torch.tensor([[node1.item()], [node2.item()]], dtype=torch.long)
        edge_index = torch.cat([edge_index, new_edge], dim=1)

    data.edge_index = edge_index
    return data

def subgraph_attack(data, poison_rate=0.20):
    edge_index = data.edge_index.clone()
    subgraph_size = int(poison_rate * data.num_nodes)

    for _ in range(subgraph_size):
        node1 = torch.randint(0, data.num_nodes, (1,))
        node2 = torch.randint(0, data.num_nodes, (1,))
        new_edge = torch.tensor([[node1.item()], [node2.item()]], dtype=torch.long)
        edge_index = torch.cat([edge_index, new_edge], dim=1)

    data.edge_index = edge_index
    return data

# Augmentation Functions
def node_level_augmentation(data):
    data.x = F.dropout(data.x, p=0.1, training=True)
    return data

def edge_level_augmentation(data):
    edge_index = data.edge_index
    mask = torch.rand(edge_index.size(1)) > 0.1
    data.edge_index = edge_index[:, mask]
    return data

def subgraph_augmentation(data, subgraph_nodes):
    edge_index = data.edge_index
    for node in subgraph_nodes:
        target = torch.randint(0, data.num_nodes, (1,))
        new_edge = torch.tensor([[node], [target]], dtype=torch.long)
        edge_index = torch.cat([edge_index, new_edge], dim=1)
    data.edge_index = edge_index
    return data

def graph_level_augmentation(data):
    noise = torch.randn_like(data.x) * 0.1
    data.x = data.x + noise
    return data

# Filtration Methods
def structural_relationship_analysis(data):
    edge_index = data.edge_index.clone()
    mask = torch.ones(edge_index.size(1), dtype=torch.bool)
    for i in range(edge_index.size(1)):
        node1, node2 = edge_index[:, i]
        if (data.x[node1] - data.x[node2]).norm() > 1.0:
            mask[i] = False
    data.edge_index = edge_index[:, mask]
    return data

def feature_manipulation_detection(data):
    for node in range(data.num_nodes):
        neighbors = data.edge_index[1][data.edge_index[0] == node]
        if neighbors.size(0) > 0:
            avg_feature = data.x[neighbors].mean(dim=0)
            if (data.x[node] - avg_feature).norm() > 0.5:
                data.x[node] = avg_feature
    return data

def edge_alteration_detection(data):
    edge_index = data.edge_index.clone()
    mask = torch.ones(edge_index.size(1), dtype=torch.bool)
    for i in range(edge_index.size(1)):
        node1, node2 = edge_index[:, i]
        if torch.abs(node1 - node2) > 10:
            mask[i] = False
    data.edge_index = edge_index[:, mask]
    return data

def pattern_based_anomaly_detection(data):
    edge_index = data.edge_index.clone()
    mask = torch.ones(edge_index.size(1), dtype=torch.bool)
    for i in range(edge_index.size(1)):
        node1, node2 = edge_index[:, i]
        if torch.abs(data.x[node1].sum() - data.x[node2].sum()) > 5.0:
            mask[i] = False
    data.edge_index = edge_index[:, mask]
    return data

# Testing each filtration method's effectiveness after poisoning and augmentations
results = {}

# Define subgraph nodes for subgraph augmentation
subgraph_nodes = torch.randperm(data.num_nodes)[:int(0.1 * data.num_nodes)]

# Run each attack, measure accuracy, then apply augmentation and filtration
attack_augment_filter_combinations = [
    ("node_level_attack", node_level_augmentation, structural_relationship_analysis, "Structural Relationship Analysis"),
    ("edge_level_attack", edge_level_augmentation, feature_manipulation_detection, "Feature Manipulation Detection"),
    ("subgraph_attack", lambda d: subgraph_augmentation(d, subgraph_nodes), edge_alteration_detection, "Edge Alteration Detection"),
    ("subgraph_attack", graph_level_augmentation, pattern_based_anomaly_detection, "Pattern-Based Anomaly Detection"),
]

for attack_fn_name, augment_fn, filter_fn, filter_name in attack_augment_filter_combinations:
    # Apply attack
    attack_fn = globals()[attack_fn_name]
    poisoned_data = attack_fn(data.clone())

    # Measure accuracy after attack
    model = GCN().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    attack_acc = []
    for epoch in range(200):
        train()
        attack_acc.append(test())
    print(f"Accuracy after {attack_fn_name}:", attack_acc[-1])

    # Apply augmentation and measure accuracy
    augmented_data = augment_fn(poisoned_data.clone())
    model = GCN().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    augmented_acc = []
    for epoch in range(200):
        train()
        augmented_acc.append(test())
    print(f"Accuracy after augmentation ({augment_fn.__name__}):", augmented_acc[-1])

    # Apply filtration method and measure final accuracy
    filtered_data = filter_fn(augmented_data.clone())
    model = GCN().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    filtered_acc = []
    for epoch in range(200):
        train()
        filtered_acc.append(test())
    print(f"Accuracy after filtration ({filter_name}):", filtered_acc[-1])

    # Store results
    results[filter_name] = {
        "poisoned_accuracy": attack_acc[-1],
        "augmented_accuracy": augmented_acc[-1],
        "filtered_accuracy": filtered_acc[-1]
    }

# Display results
print("\nFinal Results:")
for filter_name, result in results.items():
    print(f"{filter_name}:")
    print(f"  Accuracy after attack: {result['poisoned_accuracy']}")
    print(f"  Accuracy after augmentation: {result['augmented_accuracy']}")
    print(f"  Accuracy after filtration: {result['filtered_accuracy']}")


Accuracy on clean data: 0.806
Accuracy after node_level_attack: 0.819
Accuracy after augmentation (node_level_augmentation): 0.803
Accuracy after filtration (Structural Relationship Analysis): 0.812
Accuracy after edge_level_attack: 0.811
Accuracy after augmentation (edge_level_augmentation): 0.813
Accuracy after filtration (Feature Manipulation Detection): 0.814
Accuracy after subgraph_attack: 0.811
Accuracy after augmentation (<lambda>): 0.806
Accuracy after filtration (Edge Alteration Detection): 0.782
Accuracy after subgraph_attack: 0.804
Accuracy after augmentation (graph_level_augmentation): 0.81
Accuracy after filtration (Pattern-Based Anomaly Detection): 0.808

Final Results:
Structural Relationship Analysis:
  Accuracy after attack: 0.819
  Accuracy after augmentation: 0.803
  Accuracy after filtration: 0.812
Feature Manipulation Detection:
  Accuracy after attack: 0.811
  Accuracy after augmentation: 0.813
  Accuracy after filtration: 0.814
Edge Alteration Detection:
  Accura

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.transforms import NormalizeFeatures
from sklearn.metrics import accuracy_score

# Load and preprocess Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora', transform=NormalizeFeatures())
data = dataset[0]

# Define GNN model
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Initialize model, optimizer, and data
model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)
model = model.to(device)

# Function to train the model
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

# Function to test the model
def test():
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)
    acc = accuracy_score(data.y[data.test_mask].cpu(), pred[data.test_mask].cpu())
    return acc

# Train and evaluate on clean data
clean_accuracies = []
for epoch in range(200):
    train()
    acc = test()
    clean_accuracies.append(acc)
print("Accuracy on clean data:", clean_accuracies[-1])

# Poisoning Attack Functions
def node_level_attack(data, poison_rate=0.5):
    num_poison = int(poison_rate * data.num_nodes)
    for _ in range(num_poison):
        node = torch.randint(0, data.num_nodes, (1,))
        data.x[node] = torch.rand(data.x[node].shape)  # Randomly alter features
    return data

def edge_level_attack(data, poison_rate=0.5):
    num_poison = int(poison_rate * data.num_edges)
    edge_index = data.edge_index.clone()

    for _ in range(num_poison):
        node1 = torch.randint(0, data.num_nodes, (1,))
        node2 = torch.randint(0, data.num_nodes, (1,))
        new_edge = torch.tensor([[node1.item()], [node2.item()]], dtype=torch.long)
        edge_index = torch.cat([edge_index, new_edge], dim=1)

    data.edge_index = edge_index
    return data

def subgraph_attack(data, poison_rate=0.5):
    edge_index = data.edge_index.clone()
    subgraph_size = int(poison_rate * data.num_nodes)

    for _ in range(subgraph_size):
        node1 = torch.randint(0, data.num_nodes, (1,))
        node2 = torch.randint(0, data.num_nodes, (1,))
        new_edge = torch.tensor([[node1.item()], [node2.item()]], dtype=torch.long)
        edge_index = torch.cat([edge_index, new_edge], dim=1)

    data.edge_index = edge_index
    return data

# Augmentation Functions
def node_level_augmentation(data):
    data.x = F.dropout(data.x, p=0.1, training=True)
    return data

def edge_level_augmentation(data):
    edge_index = data.edge_index
    mask = torch.rand(edge_index.size(1)) > 0.1
    data.edge_index = edge_index[:, mask]
    return data

def subgraph_augmentation(data, subgraph_nodes):
    edge_index = data.edge_index
    for node in subgraph_nodes:
        target = torch.randint(0, data.num_nodes, (1,))
        new_edge = torch.tensor([[node], [target]], dtype=torch.long)
        edge_index = torch.cat([edge_index, new_edge], dim=1)
    data.edge_index = edge_index
    return data

def graph_level_augmentation(data):
    noise = torch.randn_like(data.x) * 0.1
    data.x = data.x + noise
    return data

# Filtration Methods
def structural_relationship_analysis(data):
    edge_index = data.edge_index.clone()
    mask = torch.ones(edge_index.size(1), dtype=torch.bool)
    for i in range(edge_index.size(1)):
        node1, node2 = edge_index[:, i]
        if (data.x[node1] - data.x[node2]).norm() > 1.0:
            mask[i] = False
    data.edge_index = edge_index[:, mask]
    return data

def feature_manipulation_detection(data):
    for node in range(data.num_nodes):
        neighbors = data.edge_index[1][data.edge_index[0] == node]
        if neighbors.size(0) > 0:
            avg_feature = data.x[neighbors].mean(dim=0)
            if (data.x[node] - avg_feature).norm() > 0.5:
                data.x[node] = avg_feature
    return data

def edge_alteration_detection(data):
    edge_index = data.edge_index.clone()
    mask = torch.ones(edge_index.size(1), dtype=torch.bool)
    for i in range(edge_index.size(1)):
        node1, node2 = edge_index[:, i]
        if torch.abs(node1 - node2) > 10:
            mask[i] = False
    data.edge_index = edge_index[:, mask]
    return data

def pattern_based_anomaly_detection(data):
    edge_index = data.edge_index.clone()
    mask = torch.ones(edge_index.size(1), dtype=torch.bool)
    for i in range(edge_index.size(1)):
        node1, node2 = edge_index[:, i]
        if torch.abs(data.x[node1].sum() - data.x[node2].sum()) > 5.0:
            mask[i] = False
    data.edge_index = edge_index[:, mask]
    return data

# Testing each filtration method's effectiveness after poisoning and augmentations
results = {}

# Define subgraph nodes for subgraph augmentation
subgraph_nodes = torch.randperm(data.num_nodes)[:int(0.1 * data.num_nodes)]

# Run each attack, measure accuracy, then apply augmentation and filtration
attack_augment_filter_combinations = [
    ("node_level_attack", node_level_augmentation, structural_relationship_analysis, "Structural Relationship Analysis"),
    ("edge_level_attack", edge_level_augmentation, feature_manipulation_detection, "Feature Manipulation Detection"),
    ("subgraph_attack", lambda d: subgraph_augmentation(d, subgraph_nodes), edge_alteration_detection, "Edge Alteration Detection"),
    ("subgraph_attack", graph_level_augmentation, pattern_based_anomaly_detection, "Pattern-Based Anomaly Detection"),
]

for attack_fn_name, augment_fn, filter_fn, filter_name in attack_augment_filter_combinations:
    # Apply attack
    attack_fn = globals()[attack_fn_name]
    poisoned_data = attack_fn(data.clone())

    # Measure accuracy after attack
    model = GCN().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    attack_acc = []
    for epoch in range(200):
        train()
        attack_acc.append(test())
    print(f"Accuracy after {attack_fn_name}:", attack_acc[-1])

    # Apply augmentation and measure accuracy
    augmented_data = augment_fn(poisoned_data.clone())
    model = GCN().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    augmented_acc = []
    for epoch in range(200):
        train()
        augmented_acc.append(test())
    print(f"Accuracy after augmentation ({augment_fn.__name__}):", augmented_acc[-1])

    # Apply filtration method and measure final accuracy
    filtered_data = filter_fn(augmented_data.clone())
    model = GCN().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    filtered_acc = []
    for epoch in range(200):
        train()
        filtered_acc.append(test())
    print(f"Accuracy after filtration ({filter_name}):", filtered_acc[-1])

    # Store results
    results[filter_name] = {
        "poisoned_accuracy": attack_acc[-1],
        "augmented_accuracy": augmented_acc[-1],
        "filtered_accuracy": filtered_acc[-1]
    }

# Display results
print("\nFinal Results:")
for filter_name, result in results.items():
    print(f"{filter_name}:")
    print(f"  Accuracy after attack: {result['poisoned_accuracy']}")
    print(f"  Accuracy after augmentation: {result['augmented_accuracy']}")
    print(f"  Accuracy after filtration: {result['filtered_accuracy']}")


Accuracy on clean data: 0.823
Accuracy after node_level_attack: 0.808
Accuracy after augmentation (node_level_augmentation): 0.809
Accuracy after filtration (Structural Relationship Analysis): 0.811
Accuracy after edge_level_attack: 0.801
Accuracy after augmentation (edge_level_augmentation): 0.801
Accuracy after filtration (Feature Manipulation Detection): 0.81
Accuracy after subgraph_attack: 0.806
Accuracy after augmentation (<lambda>): 0.816
Accuracy after filtration (Edge Alteration Detection): 0.815
Accuracy after subgraph_attack: 0.816
Accuracy after augmentation (graph_level_augmentation): 0.825
Accuracy after filtration (Pattern-Based Anomaly Detection): 0.813

Final Results:
Structural Relationship Analysis:
  Accuracy after attack: 0.808
  Accuracy after augmentation: 0.809
  Accuracy after filtration: 0.811
Feature Manipulation Detection:
  Accuracy after attack: 0.801
  Accuracy after augmentation: 0.801
  Accuracy after filtration: 0.81
Edge Alteration Detection:
  Accurac

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.transforms import NormalizeFeatures
from sklearn.metrics import accuracy_score

# Load and preprocess Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora', transform=NormalizeFeatures())
data = dataset[0]

# Define GNN model
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Initialize model, optimizer, and data
model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)
model = model.to(device)

# Function to train the model
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

# Function to test the model
def test():
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)
    acc = accuracy_score(data.y[data.test_mask].cpu(), pred[data.test_mask].cpu())
    return acc

# Train and evaluate on clean data
clean_accuracies = []
for epoch in range(200):
    train()
    acc = test()
    clean_accuracies.append(acc)
clean_accuracy = clean_accuracies[-1]
print("Accuracy on clean data:", clean_accuracy)

# Set the accuracy threshold to determine effectiveness
threshold = clean_accuracy * 0.9  # 90% of the clean accuracy

# Poisoning Attack Functions
def node_level_attack(data, poison_rate=0.2):
    num_poison = int(poison_rate * data.num_nodes)
    for _ in range(num_poison):
        node = torch.randint(0, data.num_nodes, (1,))
        data.x[node] = torch.rand(data.x[node].shape)  # Randomly alter features
    return data

def edge_level_attack(data, poison_rate=0.2):
    num_poison = int(poison_rate * data.num_edges)
    edge_index = data.edge_index.clone()

    for _ in range(num_poison):
        node1 = torch.randint(0, data.num_nodes, (1,))
        node2 = torch.randint(0, data.num_nodes, (1,))
        new_edge = torch.tensor([[node1.item()], [node2.item()]], dtype=torch.long)
        edge_index = torch.cat([edge_index, new_edge], dim=1)

    data.edge_index = edge_index
    return data

def subgraph_attack(data, poison_rate=0.2):
    edge_index = data.edge_index.clone()
    subgraph_size = int(poison_rate * data.num_nodes)

    for _ in range(subgraph_size):
        node1 = torch.randint(0, data.num_nodes, (1,))
        node2 = torch.randint(0, data.num_nodes, (1,))
        new_edge = torch.tensor([[node1.item()], [node2.item()]], dtype=torch.long)
        edge_index = torch.cat([edge_index, new_edge], dim=1)

    data.edge_index = edge_index
    return data

# Augmentation Functions
def node_level_augmentation(data):
    data.x = F.dropout(data.x, p=0.1, training=True)
    return data

def edge_level_augmentation(data):
    edge_index = data.edge_index
    mask = torch.rand(edge_index.size(1)) > 0.1
    data.edge_index = edge_index[:, mask]
    return data

def subgraph_augmentation(data, subgraph_nodes):
    edge_index = data.edge_index
    for node in subgraph_nodes:
        target = torch.randint(0, data.num_nodes, (1,))
        new_edge = torch.tensor([[node], [target]], dtype=torch.long)
        edge_index = torch.cat([edge_index, new_edge], dim=1)
    data.edge_index = edge_index
    return data

def graph_level_augmentation(data):
    noise = torch.randn_like(data.x) * 0.1
    data.x = data.x + noise
    return data

# Filtration Methods
def structural_relationship_analysis(data):
    edge_index = data.edge_index.clone()
    mask = torch.ones(edge_index.size(1), dtype=torch.bool)
    for i in range(edge_index.size(1)):
        node1, node2 = edge_index[:, i]
        if (data.x[node1] - data.x[node2]).norm() > 1.0:
            mask[i] = False
    data.edge_index = edge_index[:, mask]
    return data

def feature_manipulation_detection(data):
    for node in range(data.num_nodes):
        neighbors = data.edge_index[1][data.edge_index[0] == node]
        if neighbors.size(0) > 0:
            avg_feature = data.x[neighbors].mean(dim=0)
            if (data.x[node] - avg_feature).norm() > 0.5:
                data.x[node] = avg_feature
    return data

def edge_alteration_detection(data):
    edge_index = data.edge_index.clone()
    mask = torch.ones(edge_index.size(1), dtype=torch.bool)
    for i in range(edge_index.size(1)):
        node1, node2 = edge_index[:, i]
        if torch.abs(node1 - node2) > 10:
            mask[i] = False
    data.edge_index = edge_index[:, mask]
    return data

def pattern_based_anomaly_detection(data):
    edge_index = data.edge_index.clone()
    mask = torch.ones(edge_index.size(1), dtype=torch.bool)
    for i in range(edge_index.size(1)):
        node1, node2 = edge_index[:, i]
        if torch.abs(data.x[node1].sum() - data.x[node2].sum()) > 5.0:
            mask[i] = False
    data.edge_index = edge_index[:, mask]
    return data

# Testing each filtration method's effectiveness after poisoning and augmentations
results = {}

# Define subgraph nodes for subgraph augmentation
subgraph_nodes = torch.randperm(data.num_nodes)[:int(0.1 * data.num_nodes)]

# Run each attack, measure accuracy, then apply augmentation and filtration
attack_augment_filter_combinations = [
    ("node_level_attack", node_level_augmentation, structural_relationship_analysis, "Structural Relationship Analysis"),
    ("edge_level_attack", edge_level_augmentation, feature_manipulation_detection, "Feature Manipulation Detection"),
    ("subgraph_attack", lambda d: subgraph_augmentation(d, subgraph_nodes), edge_alteration_detection, "Edge Alteration Detection"),
    ("subgraph_attack", graph_level_augmentation, pattern_based_anomaly_detection, "Pattern-Based Anomaly Detection"),
]

for attack_fn_name, augment_fn, filter_fn, filter_name in attack_augment_filter_combinations:
    # Apply attack
    attack_fn = globals()[attack_fn_name]
    poisoned_data = attack_fn(data.clone())

    # Measure accuracy after attack
    model = GCN().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    attack_acc = []
    for epoch in range(200):
        train()
        attack_acc.append(test())
    print(f"Accuracy after {attack_fn_name}:", attack_acc[-1])

    # Apply augmentation and measure accuracy
    augmented_data = augment_fn(poisoned_data.clone())
    model = GCN().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    augmented_acc = []
    for epoch in range(200):
        train()
        augmented_acc.append(test())
    print(f"Accuracy after augmentation ({augment_fn.__name__}):", augmented_acc[-1])

    # Apply filtration method and measure final accuracy
    filtered_data = filter_fn(augmented_data.clone())
    model = GCN().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    filtered_acc = []
    for epoch in range(200):
        train()
        filtered_acc.append(test())
    print(f"Accuracy after filtration ({filter_name}):", filtered_acc[-1])

    # Store results and evaluate against the threshold
    final_accuracy = filtered_acc[-1]
    results[filter_name] = {
        "poisoned_accuracy": attack_acc[-1],
        "augmented_accuracy": augmented_acc[-1],
        "filtered_accuracy": final_accuracy,
        "is_effective": final_accuracy >= threshold
    }

# Display results
print("\nFinal Results:")
for filter_name, result in results.items():
    print(f"{filter_name}:")
    print(f"  Accuracy after attack: {result['poisoned_accuracy']}")
    print(f"  Accuracy after augmentation: {result['augmented_accuracy']}")
    print(f"  Accuracy after filtration: {result['filtered_accuracy']}")
    print(f"  Is effective (meets threshold): {result['is_effective']}")


Accuracy on clean data: 0.82
Accuracy after node_level_attack: 0.807
Accuracy after augmentation (node_level_augmentation): 0.817
Accuracy after filtration (Structural Relationship Analysis): 0.802
Accuracy after edge_level_attack: 0.803
Accuracy after augmentation (edge_level_augmentation): 0.806
Accuracy after filtration (Feature Manipulation Detection): 0.805
Accuracy after subgraph_attack: 0.809
Accuracy after augmentation (<lambda>): 0.803
Accuracy after filtration (Edge Alteration Detection): 0.814
Accuracy after subgraph_attack: 0.807
Accuracy after augmentation (graph_level_augmentation): 0.815
Accuracy after filtration (Pattern-Based Anomaly Detection): 0.808

Final Results:
Structural Relationship Analysis:
  Accuracy after attack: 0.807
  Accuracy after augmentation: 0.817
  Accuracy after filtration: 0.802
  Is effective (meets threshold): True
Feature Manipulation Detection:
  Accuracy after attack: 0.803
  Accuracy after augmentation: 0.806
  Accuracy after filtration: 0.

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.transforms import NormalizeFeatures
from sklearn.metrics import accuracy_score

# Load and preprocess Cora dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora', transform=NormalizeFeatures())
data = dataset[0]

# Define GNN model
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Initialize model, optimizer, and data
model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)
model = model.to(device)

# Function to train the model
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

# Function to test the model
def test():
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)
    acc = accuracy_score(data.y[data.test_mask].cpu(), pred[data.test_mask].cpu())
    return acc

# Train and evaluate on clean data
clean_accuracies = []
for epoch in range(200):
    train()
    acc = test()
    clean_accuracies.append(acc)
clean_accuracy = clean_accuracies[-1]
print("Accuracy on clean data:", clean_accuracy)

# Set the accuracy threshold to determine effectiveness
threshold = clean_accuracy * 0.9  # 90% of the clean accuracy

# Poisoning Attack Functions
def node_level_attack(data, poison_rate=0.2):
    num_poison = int(poison_rate * data.num_nodes)
    for _ in range(num_poison):
        node = torch.randint(0, data.num_nodes, (1,))
        data.x[node] = torch.rand(data.x[node].shape)  # Randomly alter features
    return data

def edge_level_attack(data, poison_rate=0.2):
    num_poison = int(poison_rate * data.num_edges)
    edge_index = data.edge_index.clone()

    for _ in range(num_poison):
        node1 = torch.randint(0, data.num_nodes, (1,))
        node2 = torch.randint(0, data.num_nodes, (1,))
        new_edge = torch.tensor([[node1.item()], [node2.item()]], dtype=torch.long)
        edge_index = torch.cat([edge_index, new_edge], dim=1)

    data.edge_index = edge_index
    return data

def subgraph_attack(data, poison_rate=0.2):
    edge_index = data.edge_index.clone()
    subgraph_size = int(poison_rate * data.num_nodes)

    for _ in range(subgraph_size):
        node1 = torch.randint(0, data.num_nodes, (1,))
        node2 = torch.randint(0, data.num_nodes, (1,))
        new_edge = torch.tensor([[node1.item()], [node2.item()]], dtype=torch.long)
        edge_index = torch.cat([edge_index, new_edge], dim=1)

    data.edge_index = edge_index
    return data

# Augmentation Functions
def node_level_augmentation(data):
    data.x = F.dropout(data.x, p=0.1, training=True)
    return data

def edge_level_augmentation(data):
    edge_index = data.edge_index
    mask = torch.rand(edge_index.size(1)) > 0.1
    data.edge_index = edge_index[:, mask]
    return data

def subgraph_augmentation(data, subgraph_nodes):
    edge_index = data.edge_index
    for node in subgraph_nodes:
        target = torch.randint(0, data.num_nodes, (1,))
        new_edge = torch.tensor([[node], [target]], dtype=torch.long)
        edge_index = torch.cat([edge_index, new_edge], dim=1)
    data.edge_index = edge_index
    return data

def graph_level_augmentation(data):
    noise = torch.randn_like(data.x) * 0.1
    data.x = data.x + noise
    return data

# Filtration Methods
def structural_relationship_analysis(data):
    edge_index = data.edge_index.clone()
    mask = torch.ones(edge_index.size(1), dtype=torch.bool)
    for i in range(edge_index.size(1)):
        node1, node2 = edge_index[:, i]
        if (data.x[node1] - data.x[node2]).norm() > 1.0:
            mask[i] = False
    data.edge_index = edge_index[:, mask]
    return data

def feature_manipulation_detection(data):
    for node in range(data.num_nodes):
        neighbors = data.edge_index[1][data.edge_index[0] == node]
        if neighbors.size(0) > 0:
            avg_feature = data.x[neighbors].mean(dim=0)
            if (data.x[node] - avg_feature).norm() > 0.5:
                data.x[node] = avg_feature
    return data

def edge_alteration_detection(data):
    edge_index = data.edge_index.clone()
    mask = torch.ones(edge_index.size(1), dtype=torch.bool)
    for i in range(edge_index.size(1)):
        node1, node2 = edge_index[:, i]
        if torch.abs(node1 - node2) > 10:
            mask[i] = False
    data.edge_index = edge_index[:, mask]
    return data

def pattern_based_anomaly_detection(data):
    edge_index = data.edge_index.clone()
    mask = torch.ones(edge_index.size(1), dtype=torch.bool)
    for i in range(edge_index.size(1)):
        node1, node2 = edge_index[:, i]
        if torch.abs(data.x[node1].sum() - data.x[node2].sum()) > 5.0:
            mask[i] = False
    data.edge_index = edge_index[:, mask]
    return data

# Testing each filtration method's effectiveness after poisoning and augmentations
results = {}

# Define subgraph nodes for subgraph augmentation
subgraph_nodes = torch.randperm(data.num_nodes)[:int(0.1 * data.num_nodes)]

# Run each attack, measure accuracy, then apply augmentation and filtration
attack_augment_filter_combinations = [
    ("node_level_attack", node_level_augmentation, structural_relationship_analysis, "Structural Relationship Analysis"),
    ("edge_level_attack", edge_level_augmentation, feature_manipulation_detection, "Feature Manipulation Detection"),
    ("subgraph_attack", lambda d: subgraph_augmentation(d, subgraph_nodes), edge_alteration_detection, "Edge Alteration Detection"),
    ("subgraph_attack", graph_level_augmentation, pattern_based_anomaly_detection, "Pattern-Based Anomaly Detection"),
]

for attack_fn_name, augment_fn, filter_fn, filter_name in attack_augment_filter_combinations:
    # Apply attack
    attack_fn = globals()[attack_fn_name]
    poisoned_data = attack_fn(data.clone())

    # Measure accuracy after attack
    model = GCN().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    attack_acc = []
    for epoch in range(200):
        train()
        attack_acc.append(test())
    attack_accuracy = attack_acc[-1]
    print(f"Accuracy after {attack_fn_name}:", attack_accuracy)

    # Calculate attack success rate
    attack_success_rate = (clean_accuracy - attack_accuracy) / clean_accuracy
    print(f"Attack Success Rate for {attack_fn_name}: {attack_success_rate * 100:.2f}%")

    # Apply augmentation and measure accuracy
    augmented_data = augment_fn(poisoned_data.clone())
    model = GCN().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    augmented_acc = []
    for epoch in range(200):
        train()
        augmented_acc.append(test())
    augmented_accuracy = augmented_acc[-1]
    print(f"Accuracy after augmentation ({augment_fn.__name__}):", augmented_accuracy)

    # Apply filtration method and measure final accuracy
    filtered_data = filter_fn(augmented_data.clone())
    model = GCN().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    filtered_acc = []
    for epoch in range(200):
        train()
        filtered_acc.append(test())
    final_accuracy = filtered_acc[-1]
    print(f"Accuracy after filtration ({filter_name}):", final_accuracy)

    # Store results and evaluate against the threshold
    results[filter_name] = {
        "poisoned_accuracy": attack_accuracy,
        "augmented_accuracy": augmented_accuracy,
        "filtered_accuracy": final_accuracy,
        "is_effective": final_accuracy >= threshold,
        "attack_success_rate": attack_success_rate * 100  # Converted to percentage
    }

# Display results
print("\nFinal Results:")
for filter_name, result in results.items():
    print(f"{filter_name}:")
    print(f"  Accuracy after attack: {result['poisoned_accuracy']}")
    print(f"  Attack Success Rate: {result['attack_success_rate']:.2f}%")
    print(f"  Accuracy after augmentation: {result['augmented_accuracy']}")
    print(f"  Accuracy after filtration: {result['filtered_accuracy']}")
    print(f"  Is effective (meets threshold): {result['is_effective']}")


Processing...
Done!


Accuracy on clean data: 0.808
Accuracy after node_level_attack: 0.818
Attack Success Rate for node_level_attack: -1.24%
Accuracy after augmentation (node_level_augmentation): 0.812
Accuracy after filtration (Structural Relationship Analysis): 0.804
Accuracy after edge_level_attack: 0.82
Attack Success Rate for edge_level_attack: -1.49%
Accuracy after augmentation (edge_level_augmentation): 0.802
Accuracy after filtration (Feature Manipulation Detection): 0.808
Accuracy after subgraph_attack: 0.815
Attack Success Rate for subgraph_attack: -0.87%
Accuracy after augmentation (<lambda>): 0.818
Accuracy after filtration (Edge Alteration Detection): 0.794
Accuracy after subgraph_attack: 0.818
Attack Success Rate for subgraph_attack: -1.24%
Accuracy after augmentation (graph_level_augmentation): 0.817
Accuracy after filtration (Pattern-Based Anomaly Detection): 0.806

Final Results:
Structural Relationship Analysis:
  Accuracy after attack: 0.818
  Attack Success Rate: -1.24%
  Accuracy after

In [3]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.transforms import NormalizeFeatures
from sklearn.metrics import accuracy_score

# Load and preprocess PubMed dataset
dataset = Planetoid(root='/tmp/PubMed', name='PubMed', transform=NormalizeFeatures())
data = dataset[0]

# Define GNN model
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Initialize model, optimizer, and data
model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)
model = model.to(device)

# Function to train the model
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

# Function to test the model
def test():
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)
    acc = accuracy_score(data.y[data.test_mask].cpu(), pred[data.test_mask].cpu())
    return acc

# Train and evaluate on clean data
clean_accuracies = []
for epoch in range(200):
    train()
    acc = test()
    clean_accuracies.append(acc)
clean_accuracy = clean_accuracies[-1]
print("Accuracy on clean data:", clean_accuracy)

# Set the accuracy threshold to determine effectiveness
threshold = clean_accuracy * 0.9  # 90% of the clean accuracy

# Poisoning Attack Functions
def node_level_attack(data, poison_rate=0.2):
    num_poison = int(poison_rate * data.num_nodes)
    for _ in range(num_poison):
        node = torch.randint(0, data.num_nodes, (1,))
        data.x[node] = torch.rand(data.x[node].shape)  # Randomly alter features
    return data

def edge_level_attack(data, poison_rate=0.2):
    num_poison = int(poison_rate * data.edge_index.size(1))
    edge_index = data.edge_index.clone()

    for _ in range(num_poison):
        node1 = torch.randint(0, data.num_nodes, (1,))
        node2 = torch.randint(0, data.num_nodes, (1,))
        new_edge = torch.tensor([[node1.item()], [node2.item()]], dtype=torch.long).to(device)
        edge_index = torch.cat([edge_index, new_edge], dim=1)

    data.edge_index = edge_index
    return data

def subgraph_attack(data, poison_rate=0.2):
    edge_index = data.edge_index.clone()
    subgraph_size = int(poison_rate * data.num_nodes)

    for _ in range(subgraph_size):
        node1 = torch.randint(0, data.num_nodes, (1,))
        node2 = torch.randint(0, data.num_nodes, (1,))
        new_edge = torch.tensor([[node1.item()], [node2.item()]], dtype=torch.long).to(device)
        edge_index = torch.cat([edge_index, new_edge], dim=1)

    data.edge_index = edge_index
    return data

# Augmentation Functions
def node_level_augmentation(data):
    data.x = F.dropout(data.x, p=0.1, training=True)
    return data

def edge_level_augmentation(data):
    edge_index = data.edge_index
    mask = torch.rand(edge_index.size(1)) > 0.1
    data.edge_index = edge_index[:, mask.to(device)]
    return data

def subgraph_augmentation(data, subgraph_nodes):
    edge_index = data.edge_index
    for node in subgraph_nodes:
        target = torch.randint(0, data.num_nodes, (1,))
        new_edge = torch.tensor([[node.item()], [target.item()]], dtype=torch.long).to(device)
        edge_index = torch.cat([edge_index, new_edge], dim=1)
    data.edge_index = edge_index
    return data

def graph_level_augmentation(data):
    noise = torch.randn_like(data.x) * 0.1
    data.x = data.x + noise
    return data

# Filtration Methods
def structural_relationship_analysis(data):
    edge_index = data.edge_index.clone()
    mask = torch.ones(edge_index.size(1), dtype=torch.bool).to(device)
    for i in range(edge_index.size(1)):
        node1, node2 = edge_index[:, i]
        if (data.x[node1] - data.x[node2]).norm() > 1.0:
            mask[i] = False
    data.edge_index = edge_index[:, mask]
    return data

def feature_manipulation_detection(data):
    for node in range(data.num_nodes):
        neighbors = data.edge_index[1][data.edge_index[0] == node]
        if neighbors.size(0) > 0:
            avg_feature = data.x[neighbors].mean(dim=0)
            if (data.x[node] - avg_feature).norm() > 0.5:
                data.x[node] = avg_feature
    return data

def edge_alteration_detection(data):
    edge_index = data.edge_index.clone()
    mask = torch.ones(edge_index.size(1), dtype=torch.bool).to(device)
    for i in range(edge_index.size(1)):
        node1, node2 = edge_index[:, i]
        if torch.abs(node1 - node2) > 10:
            mask[i] = False
    data.edge_index = edge_index[:, mask]
    return data

def pattern_based_anomaly_detection(data):
    edge_index = data.edge_index.clone()
    mask = torch.ones(edge_index.size(1), dtype=torch.bool).to(device)
    for i in range(edge_index.size(1)):
        node1, node2 = edge_index[:, i]
        if torch.abs(data.x[node1].sum() - data.x[node2].sum()) > 5.0:
            mask[i] = False
    data.edge_index = edge_index[:, mask]
    return data

# Testing each filtration method's effectiveness after poisoning and augmentations
results = {}

# Define subgraph nodes for subgraph augmentation
subgraph_nodes = torch.randperm(data.num_nodes)[:int(0.1 * data.num_nodes)].to(device)

# Run each attack, measure accuracy, then apply augmentation and filtration
attack_augment_filter_combinations = [
    ("node_level_attack", node_level_augmentation, structural_relationship_analysis, "Structural Relationship Analysis"),
    ("edge_level_attack", edge_level_augmentation, feature_manipulation_detection, "Feature Manipulation Detection"),
    ("subgraph_attack", lambda d: subgraph_augmentation(d, subgraph_nodes), edge_alteration_detection, "Edge Alteration Detection"),
    ("subgraph_attack", graph_level_augmentation, pattern_based_anomaly_detection, "Pattern-Based Anomaly Detection"),
]

for attack_fn_name, augment_fn, filter_fn, filter_name in attack_augment_filter_combinations:
    # Apply attack
    attack_fn = globals()[attack_fn_name]
    poisoned_data = attack_fn(data.clone())

    # Measure accuracy after attack
    model = GCN().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    attack_acc = []
    for epoch in range(200):
        train()
        attack_acc.append(test())
    attack_accuracy = attack_acc[-1]
    print(f"Accuracy after {attack_fn_name}:", attack_accuracy)

    # Calculate attack success rate
    attack_success_rate = (clean_accuracy - attack_accuracy) / clean_accuracy
    print(f"Attack Success Rate for {attack_fn_name}: {attack_success_rate * 100:.2f}%")

    # Apply augmentation and measure accuracy
    augmented_data = augment_fn(poisoned_data.clone())
    model = GCN().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    augmented_acc = []
    for epoch in range(200):
        train()
        augmented_acc.append(test())
    augmented_accuracy = augmented_acc[-1]
    print(f"Accuracy after augmentation ({augment_fn.__name__}):", augmented_accuracy)

    # Apply filtration method and measure final accuracy
    filtered_data = filter_fn(augmented_data.clone())
    model = GCN().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    filtered_acc = []
    for epoch in range(200):
        train()
        filtered_acc.append(test())
    final_accuracy = filtered_acc[-1]
    print(f"Accuracy after filtration ({filter_name}):", final_accuracy)

    # Store results and evaluate against the threshold
    results[filter_name] = {
        "poisoned_accuracy": attack_accuracy,
        "augmented_accuracy": augmented_accuracy,
        "filtered_accuracy": final_accuracy,
        "is_effective": final_accuracy >= threshold,
        "attack_success_rate": attack_success_rate * 100  # Converted to percentage
    }

# Display results
print("\nFinal Results:")
for filter_name, result in results.items():
    print(f"{filter_name}:")
    print(f"  Accuracy after attack: {result['poisoned_accuracy']}")
    print(f"  Attack Success Rate: {result['attack_success_rate']:.2f}%")
    print(f"  Accuracy after augmentation: {result['augmented_accuracy']}")
    print(f"  Accuracy after filtration: {result['filtered_accuracy']}")
    print(f"  Is effective (meets threshold): {result['is_effective']}")


Accuracy on clean data: 0.8
Accuracy after node_level_attack: 0.797
Attack Success Rate for node_level_attack: 0.38%
Accuracy after augmentation (node_level_augmentation): 0.794
Accuracy after filtration (Structural Relationship Analysis): 0.792
Accuracy after edge_level_attack: 0.793
Attack Success Rate for edge_level_attack: 0.88%
Accuracy after augmentation (edge_level_augmentation): 0.794
Accuracy after filtration (Feature Manipulation Detection): 0.794
Accuracy after subgraph_attack: 0.795
Attack Success Rate for subgraph_attack: 0.63%
Accuracy after augmentation (<lambda>): 0.794
Accuracy after filtration (Edge Alteration Detection): 0.792
Accuracy after subgraph_attack: 0.789
Attack Success Rate for subgraph_attack: 1.38%
Accuracy after augmentation (graph_level_augmentation): 0.79
Accuracy after filtration (Pattern-Based Anomaly Detection): 0.799

Final Results:
Structural Relationship Analysis:
  Accuracy after attack: 0.797
  Attack Success Rate: 0.38%
  Accuracy after augmen

In [5]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.transforms import NormalizeFeatures
from sklearn.metrics import accuracy_score

# Load and preprocess MUTAG dataset
dataset = TUDataset(root='/tmp/MUTAG', name='MUTAG', transform=NormalizeFeatures())
data = dataset[0]

# Define GNN model for graph classification
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, 16)
        self.conv2 = GCNConv(16, 16)
        self.fc = torch.nn.Linear(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)  # Aggregate node features for each graph
        return F.log_softmax(self.fc(x), dim=1)

# Initialize model, optimizer, and data
model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Function to train the model
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.to(device))
    loss = F.nll_loss(out, data.y.to(device))
    loss.backward()
    optimizer.step()

# Function to test the model
def test():
    model.eval()
    out = model(data.to(device))
    pred = out.argmax(dim=1)
    acc = accuracy_score(data.y.cpu(), pred.cpu())
    return acc

# Train and evaluate on clean data
clean_accuracies = []
for epoch in range(200):
    train()
    acc = test()
    clean_accuracies.append(acc)
clean_accuracy = clean_accuracies[-1]
print("Accuracy on clean data:", clean_accuracy)

# Set the accuracy threshold to determine effectiveness
threshold = clean_accuracy * 0.9  # 90% of the clean accuracy

# Poisoning Attack and Augmentation/Filtration Functions (unchanged)
# Note: Use the same functions for attacks, augmentations, and filtrations as in the original code.

# Testing each filtration method's effectiveness after poisoning and augmentations
results = {}

# Define subgraph nodes for subgraph augmentation
subgraph_nodes = torch.randperm(data.num_nodes)[:int(0.1 * data.num_nodes)].to(device)

# Run each attack, measure accuracy, then apply augmentation and filtration
attack_augment_filter_combinations = [
    ("node_level_attack", node_level_augmentation, structural_relationship_analysis, "Structural Relationship Analysis"),
    ("edge_level_attack", edge_level_augmentation, feature_manipulation_detection, "Feature Manipulation Detection"),
    ("subgraph_attack", lambda d: subgraph_augmentation(d, subgraph_nodes), edge_alteration_detection, "Edge Alteration Detection"),
    ("subgraph_attack", graph_level_augmentation, pattern_based_anomaly_detection, "Pattern-Based Anomaly Detection"),
]

for attack_fn_name, augment_fn, filter_fn, filter_name in attack_augment_filter_combinations:
    # Apply attack
    attack_fn = globals()[attack_fn_name]
    poisoned_data = attack_fn(data.clone())

    # Measure accuracy after attack
    model = GCN().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    attack_acc = []
    for epoch in range(200):
        train()
        attack_acc.append(test())
    attack_accuracy = attack_acc[-1]
    print(f"Accuracy after {attack_fn_name}:", attack_accuracy)

    # Calculate attack success rate
    attack_success_rate = (clean_accuracy - attack_accuracy) / clean_accuracy
    print(f"Attack Success Rate for {attack_fn_name}: {attack_success_rate * 100:.2f}%")

    # Apply augmentation and measure accuracy
    augmented_data = augment_fn(poisoned_data.clone())
    model = GCN().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    augmented_acc = []
    for epoch in range(200):
        train()
        augmented_acc.append(test())
    augmented_accuracy = augmented_acc[-1]
    print(f"Accuracy after augmentation ({augment_fn.__name__}):", augmented_accuracy)

    # Apply filtration method and measure final accuracy
    filtered_data = filter_fn(augmented_data.clone())
    model = GCN().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    filtered_acc = []
    for epoch in range(200):
        train()
        filtered_acc.append(test())
    final_accuracy = filtered_acc[-1]
    print(f"Accuracy after filtration ({filter_name}):", final_accuracy)

    # Store results and evaluate against the threshold
    results[filter_name] = {
        "poisoned_accuracy": attack_accuracy,
        "augmented_accuracy": augmented_accuracy,
        "filtered_accuracy": final_accuracy,
        "is_effective": final_accuracy >= threshold,
        "attack_success_rate": attack_success_rate * 100  # Converted to percentage
    }

# Display results
print("\nFinal Results:")
for filter_name, result in results.items():
    print(f"{filter_name}:")
    print(f"  Accuracy after attack: {result['poisoned_accuracy']}")
    print(f"  Attack Success Rate: {result['attack_success_rate']:.2f}%")
    print(f"  Accuracy after augmentation: {result['augmented_accuracy']}")
    print(f"  Accuracy after filtration: {result['filtered_accuracy']}")
    print(f"  Is effective (meets threshold): {result['is_effective']}")


Processing...
Done!


Accuracy on clean data: 1.0
Accuracy after node_level_attack: 1.0
Attack Success Rate for node_level_attack: 0.00%
Accuracy after augmentation (node_level_augmentation): 1.0
Accuracy after filtration (Structural Relationship Analysis): 1.0
Accuracy after edge_level_attack: 1.0
Attack Success Rate for edge_level_attack: 0.00%
Accuracy after augmentation (edge_level_augmentation): 1.0
Accuracy after filtration (Feature Manipulation Detection): 1.0
Accuracy after subgraph_attack: 1.0
Attack Success Rate for subgraph_attack: 0.00%
Accuracy after augmentation (<lambda>): 1.0
Accuracy after filtration (Edge Alteration Detection): 1.0
Accuracy after subgraph_attack: 1.0
Attack Success Rate for subgraph_attack: 0.00%
Accuracy after augmentation (graph_level_augmentation): 1.0
Accuracy after filtration (Pattern-Based Anomaly Detection): 1.0

Final Results:
Structural Relationship Analysis:
  Accuracy after attack: 1.0
  Attack Success Rate: 0.00%
  Accuracy after augmentation: 1.0
  Accuracy af